#### **EEG ADHD Classification: CNN-LSTM with Bayesian Optimization**

This notebook implements a hybrid CNN-LSTM deep learning model for ADHD classification using EEG data. The model combines convolutional neural networks for spatial feature extraction with LSTM networks for temporal pattern recognition. Hyperparameter optimization is performed using Tree-structured Parzen Estimator (TPE) algorithm via Hyperopt, with an iterative convergence-based approach to ensure robust hyperparameter selection.

#### **Key Features**
- **Dual-stream architecture**: Combines raw EEG spatial-temporal features with engineered frequency band powers
- **Iterative Bayesian Optimization**: Runs multiple BO searches until standard deviation of results converges
- **Reproducible experiments**: Seed management for consistent results across runs
- **Comprehensive evaluation**: Multiple cross-validation strategies including Leave-One-Subject-Out
- **Early stopping**: Prevents overfitting with configurable patience parameter

#### **Table of Contents**

1. [First Imports](#first-imports) - Essential libraries for data processing, ML, and deep learning
2. [Read the Processed Dataset](#read-the-processed-dataset) - Load preprocessed EEG data with frequency features
3. [Group the Data](#group-the-data) - Reshape tabular data into 3D tensors for CNN-LSTM
4. [Dataset Loading](#dataset-loading) - Custom PyTorch Dataset with dual-stream architecture
5. [Model Creation](#model-creation) - Hybrid CNN-LSTM model with fusion and classification heads
6. [First Model Training](#first-model-training) - Baseline model training with default hyperparameters
7. [Helper Functions](#helper-functions) - Utilities for model creation, evaluation, and data loading
8. [Search Space Definition](#search-space-definition) - Hyperparameter search space for Bayesian Optimization
9. [Search Objective Definition](#search-objective-definition) - Objective function with train/test split or k-fold CV
10. [Hyperparameter Search](#hyperparameter-search) - TPE search with configurable iterations and seeding
11. [Results Visualization](#results-visualization) - Iterative BO with convergence tracking and analysis
    - Convergence-based optimization with stability criteria
    - Summary visualization of all BO runs
    - Best parameter extraction and final model training
12. [Cross-Validation Experiments](#cross-validation-experiments) - Window-based k-fold and LOSOCV evaluation

---

#### **First Imports**

Import essential libraries for data manipulation, machine learning, and deep learning:
- **NumPy/Pandas**: Data processing and manipulation
- **Scikit-learn**: Train/test splitting, metrics, and label encoding
- **PyTorch**: Deep learning framework for building and training neural networks
- **Torch Optimizers**: Adam, RMSprop, and SGD for model optimization

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, RMSprop, SGD

#### **Read the Processed Dataset**

Load the preprocessed and clustered ADHD EEG dataset. This dataset contains:
- **Frequency domain features**: Power spectral density values across different frequency bands
- **Window segments**: Temporal windows extracted from continuous EEG recordings
- **Electrode channels**: Data from 19 EEG electrodes plus 7 band power features
- **Subject IDs**: For cross-validation and subject-independent testing
- **Class labels**: ADHD diagnostic categories

In [ ]:
df = pd.read_csv("./processed_clustered_adhdata.csv")

frequency_count = len(df['Frequency'].unique())
window_count = len(df['Window'].unique())
numeric_df = df.drop(['ID', 'Window'], axis=1)

display(df.head())

#### **Group the Data**

Transform the 2D tabular data into 3D tensors suitable for CNN-LSTM processing:

**Reshaping Process:**
1. Group rows by window number to reconstruct temporal segments
2. Create shape: `(WINDOW_COUNT, FREQUENCY_PER_WINDOW, ELECTRODES)`
3. Separate features (X) from labels (y)
4. Add channel dimension for CNN compatibility: `(N, freq, electrodes, 1)`

**Train/Test Split:**
- 80% training, 20% testing
- Stratified sampling to maintain class distribution
- Ensures balanced representation of all diagnostic categories

In [ ]:
# shape: (windows, frequencies, electrodes)
full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

X = full_ndarray[:, :, 2:]     # drop ID/Class columns
y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Add channel dimension (N, 1, freq, electrodes)
X_train = X_train[..., np.newaxis]   # (N, freq, electrodes, 1)
X_test  = X_test[...,  np.newaxis]

print(X_train.shape)
print("Train shape:", X_train.shape)  # (N, freq, electrodes, 1)

#### **Dataset Loading**

Custom PyTorch Dataset class for EEG data with dual-stream architecture:

**Features:**
- **EEG Raw Features**: Spatial-temporal patterns from 19 electrodes (1, 77, 19)
- **Band Power Features**: Pre-computed frequency band powers (7 features)
- **Data Augmentation Ready**: Supports future augmentation strategies
- **Batch Processing**: Efficient DataLoader integration

**Architecture Rationale:**
The dual-stream approach allows the model to learn both:
1. Fine-grained spatial-temporal patterns via CNN-LSTM
2. Engineered frequency domain features via dense layers

In [11]:
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)
        # -> (N, 1, freq, electrodes)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]          # (1, 77, 26)

        x_eeg  = x[:, :, :19]    # (1, 77, 19)
        x_band = x[0, 0, 19:]    # (7,)

        return x_eeg, x_band, self.y[idx]

train_ds = EEGDataset(X_train, y_train)
test_ds  = EEGDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

#### **Model Creation**

Hybrid CNN-LSTM architecture with dual-stream feature fusion:

**CNN Branch (Spatial Feature Extraction):**
- Conv2D layers with configurable kernels and sizes
- Average pooling for downsampling
- Dropout for regularization
- Dense layer before LSTM for dimensionality reduction

**LSTM Branch (Temporal Pattern Recognition):**
- Multi-layer LSTM for sequence modeling
- Configurable hidden size and layer depth
- Dropout between LSTM layers
- Final timestep aggregation

**Band Power Branch:**
- Dense layers for frequency domain features
- ReLU activation and dropout

**Fusion & Classification:**
- Concatenate CNN-LSTM output with band power features
- Two-layer classification head
- Supports multi-class ADHD categorization

**Training Features:**
- Early stopping with patience parameter
- Best model checkpoint restoration
- Training/validation history tracking
- Overfitting detection capabilities

In [12]:
import torch
import torch.nn as nn
import numpy as np

class EEGCNNLSTM(nn.Module):
    def __init__(self, num_band_features=7, num_classes=4,
                 cnn_kernels_1=32,
                 cnn_kernel_size_1=3,
                 cnn_kernels_2=32,
                 cnn_kernel_size_2=3,
                 cnn_dropout=0.3,
                 cnn_dense=16,
                 lstm_hidden_size=32,
                 lstm_layers=4,
                 lstm_dense=64,
                 dropout=0.3):
        super().__init__()
        
        pad1 = cnn_kernel_size_1 // 2
        self.conv1   = nn.Conv2d(1, int(cnn_kernels_1), kernel_size=cnn_kernel_size_1, padding=pad1)
        self.pool1 = nn.AvgPool2d(2)
        
        pad2 = cnn_kernel_size_2 // 2
        self.conv2 = nn.Conv2d(int(cnn_kernels_1), int(cnn_kernels_2), kernel_size=cnn_kernel_size_2, padding=pad2)
        self.cnn_dropout = nn.Dropout(cnn_dropout)

        # Compute flatten size dynamically
        with torch.no_grad():
            dummy = torch.zeros(1, 1, X_train.shape[1], 19)
            out = self._forward_cnn(dummy)   # [B, C, H, W]
            b, c, h, w = out.shape
            self.seq_len = h                      # sequence length (rows)
            self.cnn_feat_dim = c * w             # CNN features per timestep

        # Dense layer BEFORE LSTM
        self.cnn_dense = nn.Linear(self.cnn_feat_dim, int(cnn_dense))

        # Two stacked LSTM layers
        self.lstm = nn.LSTM(
            input_size=int(cnn_dense),
            hidden_size=int(lstm_hidden_size),
            num_layers=int(lstm_layers),
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0.0
        )
        # self.lstm_dense = nn.Linear(int(lstm_hidden_size), int(lstm_dense))
        self.band_fc = nn.Sequential(
            nn.Linear(num_band_features, 32),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        
        # final classifier (match your original final style: dropout + linear)
        self.classifier = nn.Sequential(
            nn.Linear(int(lstm_hidden_size) + 32, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def _forward_cnn(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.cnn_dropout(x)
        return x

    def forward(self, x_eeg, x_band):
        # 1️⃣ CNN feature extraction
        x = self._forward_cnn(x_eeg)             # [B, C, H, W]

        # 2️⃣ Prepare sequence for LSTM
        x = x.permute(0, 2, 1, 3)                 # [B, H, C, W]
        x = x.contiguous().view(x.size(0), x.size(1), -1)  # [B, H, C*W]

        # 3️⃣ Dense layer for each timestep
        x = F.relu(self.cnn_dense(x))     # [B, H, dense_size]

        # 4️⃣ Two-layer LSTM
        lstm_out, _ = self.lstm(x)                # [B, H, hidden_size]

        # 5️⃣ Use last time step (or mean/attention if preferred)
        eeg_feat = lstm_out[:, -1, :]
        # eeg_feat = lstm_out.mean(dim=1)                    # [B, hidden_size]
        # eeg_feat = self.lstm_dense(eeg_feat)

        # --- Band features ---
        band_feat = self.band_fc(x_band)        # [B, 32]

        # --- Fusion ---
        fused = torch.cat([eeg_feat, band_feat], dim=1)

        # 6️⃣ Fully connected head
        x = self.classifier(fused)

        return x

    def fit(self, train_loader, test_loader, epochs, criterion, optimizer, device, patience=100):
        best_val_loss = float('inf')
        no_improve = 0

        train_losses, train_accs = [], []
        val_losses, val_accs     = [], []

        best_state = None
        for epoch in range(epochs):
            # --- Train ---
            self.train()
            train_loss, train_correct, train_total = 0.0, 0, 0
            for xb_eeg, xb_band, yb in train_loader:
                xb_eeg = xb_eeg.to(device)
                xb_band = xb_band.to(device)
                yb = yb.to(device)
                optimizer.zero_grad()
                out = self(xb_eeg, xb_band)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * xb_eeg.size(0)
                train_correct += (out.argmax(1) == yb).sum().item()
                train_total += yb.size(0)

            train_loss /= train_total
            train_acc  = train_correct / train_total
            train_losses.append(train_loss)
            train_accs.append(train_acc)

            # --- Validate ---
            self.eval()
            val_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for xb_eeg, xb_band, yb in test_loader:
                    xb_eeg = xb_eeg.to(device)
                    xb_band = xb_band.to(device)
                    yb = yb.to(device)
                    out = self(xb_eeg, xb_band)
                    loss = criterion(out, yb)
                    val_loss += loss.item() * xb_eeg.size(0)
                    val_correct += (out.argmax(1) == yb).sum().item()
                    val_total += yb.size(0)

            val_loss /= val_total
            val_acc  = val_correct / val_total
            val_losses.append(val_loss)
            val_accs.append(val_acc)

            print(f"Epoch {epoch+1:03d} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            # if val_loss - train_loss > 0.2:
            #     print("Overfitting detected.")
            #     break

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = self.state_dict()
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print("Early stopping triggered.")
                    break

        if best_state is not None:
            self.load_state_dict(best_state)
        return {
            "train_accs": np.array(train_accs),
            "train_losses": np.array(train_losses),
            "val_accs":   np.array(val_accs),
            "val_losses": np.array(val_losses)
        }

#### **First Model Training**

Initial baseline model training with default hyperparameters:

**Purpose:**
- Establish baseline performance metrics
- Verify model architecture and data pipeline
- Identify potential issues before hyperparameter optimization

**Training Configuration:**
- 60 epochs with early stopping
- Adam optimizer with L2 regularization
- Cross-entropy loss for multi-class classification
- Automatic GPU detection and utilization

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EEGCNNLSTM().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

history = model.fit(train_loader, test_loader, epochs=60, criterion=criterion,
                    optimizer=optimizer, device=device)

In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb_eeg, xb_band, yb in test_loader:
        xb_eeg = xb_eeg.to(device)
        xb_band = xb_band.to(device)
        preds = model(xb_eeg, xb_band).argmax(1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(yb.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

print("\nTest Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds))
print(confusion_matrix(all_labels, all_preds))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_accs'], label='Training Accuracy')
plt.plot(history['val_accs'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_losses'], label='Training Loss')
plt.plot(history['val_losses'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

#### **Baseline Model Training & Comprehensive Evaluation**

Establish baseline performance with unoptimized hyperparameters for comparison:

**Purpose:**
- Create performance benchmark before hyperparameter optimization
- Understand baseline model capabilities and limitations
- Identify areas where optimization can provide improvements

**Model Configuration (Unoptimized Defaults):**
- CNN: 32 kernels per layer, 3x3 kernel size, 0.3 dropout
- LSTM: 32 hidden units, 4 layers, 64 dense units
- Optimizer: Adam with lr=1e-4, weight_decay=1e-5
- Batch size: 64
- Early stopping: patience=15

**Evaluation Strategy:**
1. Train on full train/test split
2. Comprehensive metrics: accuracy, precision, recall, F1-score
3. Confusion matrix analysis
4. Training history visualization
5. Model checkpointing for comparison

This baseline serves as the reference point for measuring the effectiveness of Bayesian hyperparameter optimization.

In [ ]:
# Define baseline hyperparameters (unoptimized)
baseline_params = {
    'cnn_kernels_1': 32,
    'cnn_kernel_size_1': 3,
    'cnn_kernels_2': 32,
    'cnn_kernel_size_2': 3,
    'cnn_dropout': 0.3,
    'cnn_dense': 16,
    'lstm_hidden_size': 32,
    'lstm_layers': 4,
    'lstm_dense': 64,
    'dropout': 0.3,
    'learning_rate': 1e-4,
    'optimizer': 'adam',
    'batch_size': 64
}

print("Baseline Model Hyperparameters:")
print("="*50)
for key, value in baseline_params.items():
    print(f"{key:20s}: {value}")
print("="*50)

In [ ]:
# Create baseline model and training components
print("\nInitializing baseline model...")
baseline_model = EEGCNNLSTM(
    cnn_kernels_1=baseline_params['cnn_kernels_1'],
    cnn_kernel_size_1=baseline_params['cnn_kernel_size_1'],
    cnn_kernels_2=baseline_params['cnn_kernels_2'],
    cnn_kernel_size_2=baseline_params['cnn_kernel_size_2'],
    cnn_dropout=baseline_params['cnn_dropout'],
    cnn_dense=baseline_params['cnn_dense'],
    lstm_hidden_size=baseline_params['lstm_hidden_size'],
    lstm_layers=baseline_params['lstm_layers'],
    lstm_dense=baseline_params['lstm_dense'],
    dropout=baseline_params['dropout']
).to(device)

# Setup optimizer and loss
baseline_optimizer = torch.optim.Adam(
    baseline_model.parameters(), 
    lr=baseline_params['learning_rate'], 
    weight_decay=1e-5
)
baseline_criterion = nn.CrossEntropyLoss()

# Create dataloaders with baseline batch size
baseline_train_loader = DataLoader(train_ds, batch_size=baseline_params['batch_size'], shuffle=True)
baseline_test_loader = DataLoader(test_ds, batch_size=baseline_params['batch_size'], shuffle=False)

# Count parameters
total_params = sum(p.numel() for p in baseline_model.parameters())
trainable_params = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Device: {device}")
print("\nModel architecture initialized successfully!")

In [ ]:
# Train baseline model
print("="*70)
print("TRAINING BASELINE MODEL (UNOPTIMIZED)")
print("="*70)
print(f"Training for max 100 epochs with early stopping (patience=15)\n")

baseline_history = baseline_model.fit(
    train_loader=baseline_train_loader,
    test_loader=baseline_test_loader,
    epochs=100,
    criterion=baseline_criterion,
    optimizer=baseline_optimizer,
    device=device,
    patience=15
)

print("\n" + "="*70)
print("BASELINE TRAINING COMPLETE")
print("="*70)
print(f"Total epochs trained: {len(baseline_history['train_losses'])}")
print(f"Best validation loss: {min(baseline_history['val_losses']):.4f}")
print(f"Best validation accuracy: {max(baseline_history['val_accs']):.4f}")
print("="*70)

In [ ]:
# Comprehensive evaluation on test set
print("\n" + "="*70)
print("BASELINE MODEL - TEST SET EVALUATION")
print("="*70)

baseline_model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for xb_eeg, xb_band, yb in baseline_test_loader:
        xb_eeg = xb_eeg.to(device)
        xb_band = xb_band.to(device)
        preds = baseline_model(xb_eeg, xb_band).argmax(1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(yb.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Calculate metrics
test_accuracy = accuracy_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
class_report = classification_report(all_labels, all_preds)

print(f"\nTest Set Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("\n" + "-"*70)
print("Classification Report:")
print("-"*70)
print(class_report)
print("-"*70)
print("Confusion Matrix:")
print("-"*70)
print(conf_matrix)
print("="*70)

In [ ]:
# Visualize training history
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(16, 5))

# Plot 1: Accuracy curves
ax1 = plt.subplot(1, 3, 1)
epochs = range(1, len(baseline_history['train_accs']) + 1)
ax1.plot(epochs, baseline_history['train_accs'], 'b-', label='Training Accuracy', linewidth=2)
ax1.plot(epochs, baseline_history['val_accs'], 'r-', label='Validation Accuracy', linewidth=2)
ax1.axhline(y=max(baseline_history['val_accs']), color='green', linestyle='--', 
            alpha=0.5, label=f'Best Val: {max(baseline_history["val_accs"]):.4f}')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Baseline Model: Accuracy Over Time', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot 2: Loss curves
ax2 = plt.subplot(1, 3, 2)
ax2.plot(epochs, baseline_history['train_losses'], 'b-', label='Training Loss', linewidth=2)
ax2.plot(epochs, baseline_history['val_losses'], 'r-', label='Validation Loss', linewidth=2)
ax2.axhline(y=min(baseline_history['val_losses']), color='green', linestyle='--', 
            alpha=0.5, label=f'Best Val: {min(baseline_history["val_losses"]):.4f}')
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title('Baseline Model: Loss Over Time', fontsize=14, fontweight='bold')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Confusion Matrix Heatmap
ax3 = plt.subplot(1, 3, 3)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True, 
            square=True, ax=ax3, cbar_kws={'label': 'Count'})
ax3.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=12, fontweight='bold')
ax3.set_title('Baseline Model: Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Baseline model training and evaluation complete!")
print(f"✓ Final test accuracy: {test_accuracy:.4f}")

In [ ]:
# Save baseline results for comparison
baseline_results = {
    'hyperparameters': baseline_params,
    'test_accuracy': float(test_accuracy),
    'best_val_accuracy': float(max(baseline_history['val_accs'])),
    'best_val_loss': float(min(baseline_history['val_losses'])),
    'epochs_trained': len(baseline_history['train_losses']),
    'total_parameters': int(total_params),
    'trainable_parameters': int(trainable_params),
    'confusion_matrix': conf_matrix.tolist(),
    'classification_report': class_report
}

# Display summary
print("\n" + "="*70)
print("BASELINE MODEL SUMMARY")
print("="*70)
print(f"Architecture: CNN-LSTM with dual-stream fusion")
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"\nTraining Results:")
print(f"  Epochs Trained: {len(baseline_history['train_losses'])}")
print(f"  Best Validation Accuracy: {max(baseline_history['val_accs']):.4f}")
print(f"  Best Validation Loss: {min(baseline_history['val_losses']):.4f}")
print(f"\nTest Set Performance:")
print(f"  Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("="*70)
print("\n✓ Baseline results saved for comparison with optimized models")

#### **Helper Functions**

Utility functions for hyperparameter optimization and model evaluation:

**Core Functions:**
- `batched()`: Iterator utility for creating batches (for cross-validation folds)
- `get_timestamp()`: Logging timestamp generation
- `get_model()`: Model instantiation with custom hyperparameters
- `get_validation()`: Comprehensive model evaluation with metrics
- `get_dataset()`: Dynamic dataset creation with configurable batch sizes

**Evaluation Metrics:**
- Accuracy score
- Classification report (precision, recall, F1-score)
- Confusion matrix
- Cross-validation support

These functions enable efficient experimentation and reproducible results across different hyperparameter configurations.

In [9]:
import itertools

def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 2) → AB CD EF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(itertools.islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import time
import datetime

def get_timestamp():
    ts = time.time()
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

def get_model(params):
    model = EEGCNNLSTM(
        cnn_kernels_1=params['cnn_kernels_1'],
        cnn_kernel_size_1=params['cnn_kernel_size_1'],
        cnn_kernels_2=params['cnn_kernels_2'],
        cnn_dropout=float(params['cnn_dropout']),
        cnn_dense=params['cnn_dense'],
        lstm_hidden_size=params['lstm_hidden_size'],
        lstm_layers=params['lstm_layers'],
        lstm_dense=params['lstm_dense'],
        dropout=float(params['cnn_dropout']),  # use cnn_dropout as a simple shared dropout param
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    if params['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=1e-4)
    elif params['optimizer'] == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=params['learning_rate'], weight_decay=1e-4)
    else:
        optimizer = optim.SGD(model.parameters(), lr=params['learning_rate'], momentum=0.9, weight_decay=1e-4)

    return model, criterion, optimizer

def get_validation(model, data_loader, device, matrix=True):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb_eeg, xb_band, yb in data_loader:
            xb_eeg = xb_eeg.to(device)
            xb_band = xb_band.to(device)
            preds = model(xb_eeg, xb_band).argmax(1).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(yb.numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds) if matrix else None

    return acc, report, conf_matrix

def get_dataset(df, is_train=False, batch_size=36):
    frequency_count = len(df['Frequency'].unique())
    window_count = len(df['Window'].unique())
    numeric_df = df.drop(['ID', 'Window'], axis=1)

    # shape: (windows, freqs, features)
    full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

    X = full_ndarray[:, :, 2:]     # drop ID/Class columns
    y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

    # Add channel dimension (N, 1, freq, electrodes)
    X = X[..., np.newaxis]          # (N, freq, electrodes, 1)

    print(X.shape)

    return DataLoader(EEGDataset(X, y), batch_size=batch_size, shuffle=is_train)

#### **Search Space Definition**

Hyperparameter search space for Bayesian Optimization using Hyperopt:

**CNN Architecture:**
- `cnn_kernels_1/2`: Number of convolutional filters [16, 32, 48, 64, 96]
- `cnn_kernel_size_1/2`: Kernel dimensions [3x3, 5x5]
- `cnn_dropout`: Regularization rate [0.0 - 0.7]
- `cnn_dense`: Dense layer size before LSTM [32, 64, 128, 256]

**LSTM Architecture:**
- `lstm_hidden_size`: Hidden state dimensions [32, 64, 96, 128]
- `lstm_layers`: Number of stacked LSTM layers [1-6]
- `lstm_dense`: Dense layer size after LSTM [32, 64, 128, 256]

**Training Configuration:**
- `learning_rate`: Log-uniform distribution [1e-5, 1e-2]
- `optimizer`: Choice of Adam, RMSprop, or SGD
- `batch_size`: Samples per batch [32, 36, 48, 64, 80, 96]

**Optimization Strategy:**
Tree-structured Parzen Estimator (TPE) algorithm balances exploration and exploitation to efficiently search the hyperparameter space.

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK

# -------------------------
# Hyperopt search space
# -------------------------
space = {
    'cnn_kernels_1'    : hp.choice('cnn_kernels_1', [16, 32, 48, 64]),
    'cnn_kernel_size_1': hp.choice('cnn_kernel_size_1', [3, 5]),
    'cnn_kernels_2'    : hp.choice('cnn_kernels_2', [16, 32, 64, 96]),
    'cnn_kernel_size_2': hp.choice('cnn_kernel_size_2', [3, 5]),
    'cnn_dropout'      : hp.uniform('cnn_dropout', 0.0, 0.7),
    'cnn_dense'        : hp.choice('cnn_dense', [32, 64, 128, 256]),
    'lstm_hidden_size' : hp.choice('lstm_hidden_size', [32, 64, 96, 128]),
    'lstm_layers'      : hp.choice('lstm_layers', [1, 2, 3, 4, 5, 6]),
    'lstm_dense'       : hp.choice('lstm_dense', [32, 64, 128, 256]),
    'learning_rate'    : hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
    'optimizer'        : hp.choice('optimizer', ['adam', 'rmsprop', 'sgd']),
    'batch_size'       : hp.choice('batch_size', [32, 36, 48, 64, 80, 96])
}

#### **Search Objective Definition**

Objective function for Hyperopt optimization with two strategies:

### Strategy 1: K-Fold Cross-Validation (Commented)
**Approach:**
- Subject-stratified K-fold cross-validation
- Cyclic fold generation for balanced distribution
- Score calculation: `mean_loss + variance_of_tail_losses`
- More robust but computationally expensive

**Metrics:**
- Mean validation loss across all folds
- Variance of final validation losses (stability measure)
- Combined score penalizes both high loss and instability

### Strategy 2: Single Train/Test Split (Active)
**Approach:**
- Uses predefined train/test split
- Faster iteration for initial hyperparameter search
- Early stopping with patience=10

**Return Value:**
- Minimizes best validation loss
- Includes training history for analysis
- STATUS_OK for successful trials

**Note:** The objective function is called by Hyperopt's `fmin()` and should return a dictionary with 'loss' and 'status' keys.

In [12]:
import itertools

# -------------------------
# k-Fold CV for Hyperopt
# -------------------------
def objective(params):
    print("Trial params:", params)

    criterion = nn.CrossEntropyLoss()
    unique_subjects = df['ID'].unique()
    losses = []
    variances = []
    batch_size = params['batch_size']

    K_FOLDS = 5
    fold_size = len(unique_subjects) // K_FOLDS

    cyclic = itertools.cycle(unique_subjects)
    batched_cyclic = batched(cyclic, n=fold_size)
    folds = itertools.islice(batched_cyclic, K_FOLDS)

    for i, fold in enumerate(folds):
        print(f"Starting fold {i + 1}/{K_FOLDS}")

        train_df = df[~df['ID'].isin(fold)]
        test_df  = df[df['ID'].isin(fold)]

        print(train_df.shape, test_df.shape)

        train_loader = get_dataset(train_df, batch_size=batch_size)
        test_loader  = get_dataset(test_df, batch_size=batch_size)
        model, criterion, optimizer = get_model(params)

        # Train with modest epochs; early stopping inside fit handles rest
        history = model.fit(
            train_loader=train_loader,
            test_loader=test_loader,
            epochs=60,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            patience=15
        )

        acc, *_ = get_validation(model, test_loader, device)
        loss = history['val_losses']
        mean_loss = np.min(loss)
        losses.append(mean_loss)

        last_5_or_less = history["val_losses"]
        last_5_or_less = last_5_or_less[-min(len(last_5_or_less), 5):]
        variance = np.var(last_5_or_less) if len(last_5_or_less) > 1 else 1
        variances.append(variance)

        print(f"Fold {i + 1} Accuracy:", acc)

    loss = np.mean(losses)
    tail_variance = np.var(variances)
    print(variances)
    score = loss + tail_variance

    print(f"k-Fold CV Mean Loss: {loss:.4f} ± {np.std(losses):.4f}")
    print(f"k-Fold CV Tail Variance: {tail_variance:.4f}")

    # Hyperopt minimizes -> return negative accuracy
    return {'loss': score, 'status': STATUS_OK, 'attachments': {'history': history}}


In [18]:
# -------------------------
# Single Objective for Hyperopt
# -------------------------
def objective(params):
    print("Trial params:", params)

    # build dataloaders from the existing train_ds/test_ds in this session
    train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=params['batch_size'], shuffle=False)

    # create model (note we pass dropout into lstm dropout and cnn dropout)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=60,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=10
    )

    best_val_loss = float(np.min(history['val_losses'])) if len(history['val_losses']) > 0 else 0.0

    # Hyperopt minimizes -> return negative accuracy
    return {'loss': best_val_loss, 'status': STATUS_OK, 'attachments': {'history': history, 'best_val_loss': best_val_loss}}


#### **Hyperparameter Search**

Execute Tree-structured Parzen Estimator (TPE) search for optimal hyperparameters:

**Search Configuration:**
- `max_evals`: Number of trials (default: 30, increase for thorough search)
- Algorithm: TPE (adaptive Bayesian optimization)
- Tracks all trials in Hyperopt Trials object

**Process:**
1. Initialize trials tracking object
2. Run TPE algorithm over search space
3. Convert raw indices to interpretable values
4. Save best parameters to JSON

**Output:**
- **Best hyperparameters**: Both raw indices and interpreted values
- **Search duration**: Total optimization time
- **Parameters JSON**: Serialized configuration for model reproduction

**Multiple Runs:**
The second cell executes 10 independent search runs to:
- Assess hyperparameter sensitivity
- Identify robust configurations
- Build ensemble of candidate models

In [ ]:
from hyperopt import Trials, fmin

def hyperparameter_search(max_evals=30, seed=None):
    trials = Trials()
    
    # Create seeded random state for Hyperopt
    rstate = np.random.RandomState(seed) if seed is not None else None
    
    print("Starting TPE search...")
    if seed is not None:
        print(f"Using seed: {seed}")
    t0 = time.time()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,   # increase for more thorough search
        trials=trials,
        rstate=rstate  # CRITICAL: seed Hyperopt's random state
    )
    
    print("Best hyperparameters:", best)
    t1 = time.time()
    duration = t1 - t0
    print(f"TPE search finished in {duration:.2f} seconds")
    print("Best (raw indices):", best)
    
    # Convert choice indices back to values for readability:
    def choice_value(key, val):
        mapping = {
            'cnn_kernels_1': [16, 32, 48, 64],
            'cnn_kernel_size_1': [3, 5],
            'cnn_kernels_2': [16, 32, 64, 96],
            'cnn_kernel_size_2': [3, 5],
            'cnn_dense': [32, 64, 128, 256],
            'lstm_hidden_size': [32, 64, 96, 128],
            'lstm_layers': [1, 2, 3, 4, 5, 6],
            'lstm_dense': [32, 64, 128, 256],
            'optimizer': ['adam', 'rmsprop', 'sgd'],
            'batch_size': [32, 36, 48, 64, 80, 96]
        }
        return mapping[key][int(val)] if key in mapping else val
    
    readable = {k: choice_value(k, v) if k in ['cnn_kernels_1','cnn_kernel_size_1','cnn_kernels_2',
                                               'cnn_kernel_size_2', 'cnn_dense','lstm_hidden_size',
                                               'lstm_layers','lstm_dense','optimizer','batch_size'] else v
                for k,v in best.items()}
    print("Best (interpreted):", readable)

    params = dict(readable)
    params['cnn_kernels_1'] = int(params['cnn_kernels_1'])
    params['cnn_kernel_size_1'] = int(params['cnn_kernel_size_1'])
    params['cnn_kernels_2'] = int(params['cnn_kernels_2'])
    params['cnn_kernel_size_2'] = int(params['cnn_kernel_size_2'])
    params['cnn_dense'] = int(params['cnn_dense'])
    params['lstm_hidden_size'] = int(params['lstm_hidden_size'])
    params['lstm_layers'] = int(params['lstm_layers'])
    params['lstm_dense'] = int(params['lstm_dense'])
    params['batch_size'] = int(params['batch_size'])
    params['cnn_dropout'] = float(params['cnn_dropout'])
    params['dropout'] = float(params['cnn_dropout'])

    return trials, params

In [ ]:
trials, params = hyperparameter_search()
with open("best_parameters.json", "w+") as f:
    import json
    json.dump(params, f, indent=4)

In [ ]:
trial_results = []

for i in range(10):
    trials, params = hyperparameter_search()
    trial_results.append({"trials": trials, "params": params})

In [ ]:
import json
import numpy as np
from datetime import datetime


def to_json_safe(obj):
    """Recursively convert objects to JSON-serializable types."""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    if isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    if isinstance(obj, datetime):
        return obj.isoformat()
    if isinstance(obj, dict):
        return {k: to_json_safe(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_json_safe(v) for v in obj]
    return obj


def serialize_trial(trial):
    """Extract only JSON-safe and meaningful parts of a Hyperopt Trial."""
    print(trial.attachments)
    return {
        "attachments": to_json_safe(trial.attachments)
    }


# ===============================
# DROP-IN REPLACEMENT STARTS HERE
# ===============================

json_results = []

for entry in trial_results:
    json_results.append({
        "params": to_json_safe(entry.get("params")),
        "trial": serialize_trial(entry.get("trials")),
    })

with open("all_trials.json", "w", encoding="utf-8") as f:
    json.dump(
        {"results": json_results},
        f,
        indent=4,
        ensure_ascii=False
    )

#### **Results Visualization**

Comprehensive hyperparameter optimization with iterative Bayesian Optimization and convergence tracking:

---

### **Iterative Convergence-Based Bayesian Optimization**

**Multi-Run Strategy:**
- Starts with 3 initial BO searches with different random seeds
- Runs 2 additional iterations and recalculates standard deviation
- Continues until std dev stabilizes (relative change < 0.1%)
- Tracks convergence to ensure robust hyperparameter selection

**Reproducibility:**
- Each BO run uses a unique sequential seed (base_seed + run_index)
- Seeds control PyTorch, NumPy, Python random states, and Hyperopt's TPE sampler
- Full seed tracking for experiment reproduction

**Stability Criteria:**
- Convergence: `|current_std - previous_std| / previous_std < 0.001`
- Reduces sensitivity to random initialization
- Ensures identified hyperparameters are stable across multiple searches

**Safety Measures:**
- Maximum limit of 20 total BO runs
- Prevents infinite loops while allowing thorough exploration

---

### **Comprehensive Visualization and Analysis**

**Performance Summary:**
- Table of all BO runs with seeds and validation losses
- Ranking of configurations by performance
- Statistical analysis: mean, std dev, min/max, range

**Convergence Plots:**
- Best validation loss across all runs (bar chart with annotations)
- Standard deviation evolution over iterations
- Percentage change annotations between measurements
- Color-coded convergence indicators (green when < 0.1% change)

**Visual Highlights:**
- Best performing run highlighted in green
- Mean and minimum loss reference lines
- Convergence threshold visualization

---

### **Best Parameters Extraction**

**Selection Process:**
- Systematically searches across all BO runs
- Identifies trial with lowest validation loss
- Extracts complete hyperparameter configuration

**Output:**
- Best run index and seed for reproducibility
- Complete hyperparameter dictionary
- Best validation loss achieved
- Saves to `best_parameters.json` for deployment

---

### **Final Model Training with Optimal Hyperparameters**

**Evaluation:**
- Trains model with best discovered hyperparameters
- Reports test set accuracy, classification report, confusion matrix
- Visualizes training history (accuracy and loss curves)
- Confirms model generalization performance

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def run_bo_with_seed(seed, max_evals=30):
    """Run Bayesian Optimization with a specific seed."""
    print(f"\n{'='*60}")
    print(f"Running BO with seed: {seed}")
    print(f"{'='*60}")
    
    set_seed(seed)
    # CRITICAL: Pass seed to hyperparameter_search to seed Hyperopt
    trials, params = hyperparameter_search(max_evals=max_evals, seed=seed)
    
    # Extract best validation loss from this run
    best_val_loss = float('inf')
    for trial in trials.trials:
        if trial['result']['status'] == 'ok':
            loss = trial['result']['loss']
            if loss < best_val_loss:
                best_val_loss = loss
    
    return {
        'seed': seed,
        'trials': trials,
        'params': params,
        'best_val_loss': best_val_loss
    }

# Initialize tracking variables
all_bo_runs = []
std_devs = []
run_counts = []  # Track number of runs for each std_dev calculation
base_seed = 42

print("Starting iterative Bayesian Optimization with convergence tracking...")
print(f"Convergence criterion: Standard deviation change < 0.1%\n")

# Initial 3 runs
print("Phase 1: Running initial 3 BO searches...")
for i in range(3):
    seed = base_seed + i
    result = run_bo_with_seed(seed, max_evals=30)
    all_bo_runs.append(result)
    
    print(f"\nRun {i+1}/3 completed:")
    print(f"  Seed: {seed}")
    print(f"  Best Val Loss: {result['best_val_loss']:.6f}")

# Calculate std dev only after all 3 initial runs
best_losses = [r['best_val_loss'] for r in all_bo_runs]
current_std = np.std(best_losses)
std_devs.append(current_std)
run_counts.append(len(all_bo_runs))

print(f"\nPhase 1 Summary:")
print(f"  Total Runs: {len(all_bo_runs)}")
print(f"  Std Dev: {current_std:.6f}")

# Iterative refinement
iteration = 2
converged = False
prev_std = std_devs[-1]

while not converged:
    print(f"\n{'='*60}")
    print(f"Phase {iteration}: Running 2 additional BO searches...")
    print(f"{'='*60}")
    
    # Run 2 more BO searches
    for i in range(2):
        seed = base_seed + len(all_bo_runs)
        result = run_bo_with_seed(seed, max_evals=30)
        all_bo_runs.append(result)
        
        print(f"\nRun {len(all_bo_runs)} completed:")
        print(f"  Seed: {seed}")
        print(f"  Best Val Loss: {result['best_val_loss']:.6f}")
    
    # Calculate new std dev after both runs complete
    best_losses = [r['best_val_loss'] for r in all_bo_runs]
    current_std = np.std(best_losses)
    std_devs.append(current_std)
    run_counts.append(len(all_bo_runs))
    
    # Check convergence
    std_change = abs(current_std - prev_std) / prev_std if prev_std > 0 else 1.0
    
    print(f"\n--- Convergence Check ---")
    print(f"Total Runs:       {len(all_bo_runs)}")
    print(f"Previous Std Dev: {prev_std:.6f}")
    print(f"Current Std Dev:  {current_std:.6f}")
    print(f"Relative Change:  {std_change*100:.4f}%")
    
    if std_change < 0.001:  # 0.1% threshold
        print(f"\n✓ Convergence achieved! Std dev change < 0.1%")
        converged = True
    else:
        print(f"\n→ Not converged yet, continuing...")
        prev_std = current_std
        iteration += 1
        
        # Safety limit
        if len(all_bo_runs) >= 20:
            print(f"\n⚠ Reached maximum of 20 runs, stopping.")
            converged = True

print(f"\n{'='*60}")
print(f"Optimization Complete!")
print(f"{'='*60}")
print(f"Total BO runs: {len(all_bo_runs)}")
print(f"Final Std Dev: {std_devs[-1]:.6f}")
print(f"Std Dev tracked at runs: {run_counts}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create summary table
summary_data = []
for i, run in enumerate(all_bo_runs):
    summary_data.append({
        'Run': i + 1,
        'Seed': run['seed'],
        'Best Val Loss': run['best_val_loss'],
        'Rank': 0  # Will be filled after sorting
    })

# Sort by best val loss and assign ranks
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Best Val Loss')
summary_df['Rank'] = range(1, len(summary_df) + 1)
summary_df = summary_df.sort_values('Run')  # Resort by run order

print("\n" + "="*70)
print("SUMMARY OF BAYESIAN OPTIMIZATION RUNS")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

# Calculate statistics
mean_loss = summary_df['Best Val Loss'].mean()
std_loss = summary_df['Best Val Loss'].std()
min_loss = summary_df['Best Val Loss'].min()
max_loss = summary_df['Best Val Loss'].max()

print(f"\nStatistics:")
print(f"  Mean Best Loss: {mean_loss:.6f}")
print(f"  Std Dev:        {std_loss:.6f}")
print(f"  Min Loss:       {min_loss:.6f}")
print(f"  Max Loss:       {max_loss:.6f}")
print(f"  Range:          {max_loss - min_loss:.6f}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Best validation losses across runs
ax1 = axes[0]
runs = summary_df['Run'].values
losses = summary_df['Best Val Loss'].values
colors = ['#2ecc71' if loss == min_loss else '#3498db' for loss in losses]

bars = ax1.bar(runs, losses, color=colors, alpha=0.7, edgecolor='black', linewidth=1.2)
ax1.axhline(y=mean_loss, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_loss:.4f}')
ax1.axhline(y=min_loss, color='green', linestyle='--', linewidth=2, alpha=0.5, label=f'Best: {min_loss:.4f}')

ax1.set_xlabel('BO Run', fontsize=12, fontweight='bold')
ax1.set_ylabel('Best Validation Loss', fontsize=12, fontweight='bold')
ax1.set_title('Best Validation Loss per BO Run', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)
ax1.set_xticks(runs)

# Annotate the best run
best_run_idx = summary_df['Best Val Loss'].idxmin()
best_run = summary_df.loc[best_run_idx, 'Run']
best_loss = summary_df.loc[best_run_idx, 'Best Val Loss']
ax1.annotate(f'Best\n{best_loss:.4f}', 
             xy=(best_run, best_loss),
             xytext=(best_run, best_loss + (max_loss - min_loss) * 0.1),
             ha='center',
             fontsize=9,
             fontweight='bold',
             color='darkgreen',
             arrowprops=dict(arrowstyle='->', color='darkgreen', lw=1.5))

# Plot 2: Standard deviation convergence
ax2 = axes[1]

# Use the tracked run_counts for accurate x-axis
ax2.plot(run_counts, std_devs, marker='o', linewidth=2.5, markersize=8, 
         color='#e74c3c', markerfacecolor='white', markeredgewidth=2)
ax2.set_xlabel('Number of BO Runs', fontsize=12, fontweight='bold')
ax2.set_ylabel('Std Dev of Best Losses', fontsize=12, fontweight='bold')
ax2.set_title('Convergence of Standard Deviation', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add convergence threshold line
if len(std_devs) > 1:
    converged_std = std_devs[-1]
    ax2.axhline(y=converged_std, color='green', linestyle='--', linewidth=2, 
                alpha=0.5, label=f'Final: {converged_std:.4f}')
    ax2.legend(fontsize=10)

# Annotate percentage changes between consecutive measurements
for i in range(1, len(std_devs)):
    pct_change = abs(std_devs[i] - std_devs[i-1]) / std_devs[i-1] * 100 if std_devs[i-1] > 0 else 0
    mid_x = (run_counts[i] + run_counts[i-1]) / 2
    mid_y = (std_devs[i] + std_devs[i-1]) / 2
    
    # Color code: red if change > 0.1%, green if <= 0.1%
    box_color = 'lightgreen' if pct_change <= 0.1 else 'yellow'
    
    ax2.annotate(f'{pct_change:.2f}%', 
                xy=(mid_x, mid_y),
                fontsize=8,
                ha='center',
                bbox=dict(boxstyle='round,pad=0.3', facecolor=box_color, alpha=0.5, edgecolor='gray'))

plt.tight_layout()
plt.show()

print(f"\n✓ Visualization complete.")
print(f"\nConvergence History:")
for i, (runs, std) in enumerate(zip(run_counts, std_devs)):
    if i == 0:
        print(f"  After {runs} runs: Std Dev = {std:.6f}")
    else:
        prev_std = std_devs[i-1]
        pct_change = abs(std - prev_std) / prev_std * 100 if prev_std > 0 else 0
        status = "✓ Converged" if pct_change < 0.1 else "→ Continuing"
        print(f"  After {runs} runs: Std Dev = {std:.6f} (Change: {pct_change:.2f}%) {status}")

In [ ]:
import json

def extract_best_params_from_runs(all_runs):
    """Extract parameters from the best performing BO run."""
    best_overall_loss = float('inf')
    best_params = None
    best_run_index = -1
    best_seed = None
    
    for run_idx, run_data in enumerate(all_runs):
        if run_data['best_val_loss'] < best_overall_loss:
            best_overall_loss = run_data['best_val_loss']
            best_params = run_data['params']
            best_run_index = run_idx + 1
            best_seed = run_data['seed']
    
    if best_params:
        print(f"\n{'='*70}")
        print("BEST PERFORMING CONFIGURATION")
        print(f"{'='*70}")
        print(f"Run Index:        {best_run_index}/{len(all_runs)}")
        print(f"Seed:             {best_seed}")
        print(f"Best Val Loss:    {best_overall_loss:.6f}")
        print(f"\nHyperparameters:")
        print(json.dumps(best_params, indent=4))
        print(f"{'='*70}")
        
        # Save best parameters
        output_data = {
            'run_index': best_run_index,
            'seed': best_seed,
            'best_val_loss': float(best_overall_loss),
            'parameters': best_params
        }
        
        with open("best_parameters.json", "w") as f:
            json.dump(output_data, f, indent=4)
        
        print(f"\n✓ Best parameters saved to 'best_parameters.json'")
        
        return best_params
    else:
        print("Error: No valid parameters found.")
        return None

# Extract best parameters from all BO runs
best_params = extract_best_params_from_runs(all_bo_runs)

In [ ]:
# build dataloaders from the existing train_ds/test_ds in this session
train_loader = DataLoader(train_ds, batch_size=best_params['batch_size'], shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=best_params['batch_size'], shuffle=False)

# create model (note we pass dropout into lstm dropout and cnn dropout)
model, criterion, optimizer = get_model(best_params)

# Train with modest epochs; early stopping inside fit handles rest
history = model.fit(
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=60,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    patience=10
)
acc, report, matrix = get_validation(model, test_loader, device)

print("\nval Accuracy:", acc)
print(report)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_accs'], label='Training Accuracy')
plt.plot(history['val_accs'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_losses'], label='Training Loss')
plt.plot(history['val_losses'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

#### **Model Architecture Visualization with TorchViz**

Visualize the computational graph structure of both models to understand:
- Forward pass data flow and operations
- Layer connectivity and tensor transformations
- Architectural differences between baseline and optimized models
- Parameter sharing and gradient flow paths

**TorchViz** generates graphical representations of PyTorch computational graphs, making it easy to:
- Debug model architecture
- Identify bottlenecks or unnecessary operations
- Compare different architectural configurations
- Understand how hyperparameter changes affect model structure

In [ ]:
# Install torchviz if not already available
try:
    from torchviz import make_dot
    print("✓ torchviz already installed")
except ImportError:
    print("Installing torchviz...")
    import sys
    !{sys.executable} -m pip install torchviz graphviz -q
    from torchviz import make_dot
    print("✓ torchviz installed successfully")

from torchviz import make_dot
import graphviz

print("✓ TorchViz ready for model visualization")

In [ ]:
# Get a sample from test set for graph generation
baseline_model.eval()
model.eval()

# Get first batch from test loader
sample_batch = next(iter(baseline_test_loader))
sample_x_eeg = sample_batch[0][0:1].to(device)  # First sample only
sample_x_band = sample_batch[1][0:1].to(device)
sample_label = sample_batch[2][0].item()

print(f"✓ Sample selected for visualization (label: {sample_label})")
print(f"  EEG shape: {sample_x_eeg.shape}")
print(f"  Band features shape: {sample_x_band.shape}")

In [ ]:
# Generate computational graph for baseline model
print("Generating computational graph for BASELINE model...")
print("="*70)

# Forward pass to create computation graph
baseline_output = baseline_model(sample_x_eeg, sample_x_band)

# Create visualization
baseline_graph = make_dot(
    baseline_output.mean(), 
    params=dict(baseline_model.named_parameters()),
    show_attrs=True,
    show_saved=False
)

# Customize graph appearance
baseline_graph.graph_attr.update({
    'rankdir': 'TB',  # Top to Bottom
    'size': '12,16',
    'dpi': '150',
    'bgcolor': 'white',
    'label': 'Baseline (Unoptimized) CNN-LSTM Model Architecture',
    'labelloc': 't',
    'fontsize': '20',
    'fontname': 'Arial Bold'
})

baseline_graph.node_attr.update({
    'style': 'filled',
    'fillcolor': 'lightblue',
    'fontname': 'Arial',
    'fontsize': '10'
})

# Render and display
baseline_graph.render('baseline_model_graph', format='png', cleanup=True)
print("✓ Baseline model graph saved as 'baseline_model_graph.png'")

# Display in notebook
from IPython.display import Image, display
display(Image('baseline_model_graph.png'))

In [ ]:
# Generate computational graph for optimized model
print("\nGenerating computational graph for OPTIMIZED model...")
print("="*70)

# Forward pass to create computation graph
optimized_output = model(sample_x_eeg, sample_x_band)

# Create visualization
optimized_graph = make_dot(
    optimized_output.mean(), 
    params=dict(model.named_parameters()),
    show_attrs=True,
    show_saved=False
)

# Customize graph appearance
optimized_graph.graph_attr.update({
    'rankdir': 'TB',  # Top to Bottom
    'size': '12,16',
    'dpi': '150',
    'bgcolor': 'white',
    'label': 'Optimized CNN-LSTM Model Architecture',
    'labelloc': 't',
    'fontsize': '20',
    'fontname': 'Arial Bold'
})

optimized_graph.node_attr.update({
    'style': 'filled',
    'fillcolor': 'lightcoral',
    'fontname': 'Arial',
    'fontsize': '10'
})

# Render and display
optimized_graph.render('optimized_model_graph', format='png', cleanup=True)
print("✓ Optimized model graph saved as 'optimized_model_graph.png'")

# Display in notebook
display(Image('optimized_model_graph.png'))

In [ ]:
# Detailed Architecture and Parameter Comparison
print("\n" + "="*70)
print("DETAILED ARCHITECTURE & PARAMETER COMPARISON")
print("="*70)

def get_layer_info(model, model_name):
    """Extract detailed layer information from model."""
    layer_info = []
    
    for name, module in model.named_modules():
        if len(list(module.children())) == 0 and name:  # Leaf modules only
            params = sum(p.numel() for p in module.parameters())
            trainable = sum(p.numel() for p in module.parameters() if p.requires_grad)
            
            # Get layer type and shape info
            layer_type = module.__class__.__name__
            
            # Get specific attributes based on layer type
            details = ""
            if isinstance(module, nn.Conv2d):
                details = f"in={module.in_channels}, out={module.out_channels}, kernel={module.kernel_size}"
            elif isinstance(module, nn.Linear):
                details = f"in={module.in_features}, out={module.out_features}"
            elif isinstance(module, nn.LSTM):
                details = f"input={module.input_size}, hidden={module.hidden_size}, layers={module.num_layers}"
            elif isinstance(module, nn.Dropout):
                details = f"p={module.p}"
            elif isinstance(module, nn.AvgPool2d):
                details = f"kernel={module.kernel_size}"
            
            layer_info.append({
                'name': name,
                'type': layer_type,
                'details': details,
                'total_params': params,
                'trainable_params': trainable
            })
    
    return layer_info

# Get layer information for both models
baseline_layers = get_layer_info(baseline_model, "Baseline")
optimized_layers = get_layer_info(model, "Optimized")

# Create comparison DataFrame
import pandas as pd

baseline_df = pd.DataFrame(baseline_layers)
optimized_df = pd.DataFrame(optimized_layers)

print("\n" + "-"*70)
print("BASELINE (UNOPTIMIZED) MODEL ARCHITECTURE")
print("-"*70)
print(baseline_df.to_string(index=False))
print(f"\nTotal Parameters: {baseline_df['total_params'].sum():,}")
print(f"Trainable Parameters: {baseline_df['trainable_params'].sum():,}")

print("\n" + "-"*70)
print("OPTIMIZED MODEL ARCHITECTURE")
print("-"*70)
print(optimized_df.to_string(index=False))
print(f"\nTotal Parameters: {optimized_df['total_params'].sum():,}")
print(f"Trainable Parameters: {optimized_df['trainable_params'].sum():,}")

print("\n" + "="*70)

In [ ]:
# Visual Comparison of Key Architectural Differences
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(18, 10))

# Extract key hyperparameters for comparison
baseline_hp = baseline_params
optimized_hp = best_params

# Prepare comparison data
comparison_metrics = {
    'CNN Kernels L1': (baseline_hp['cnn_kernels_1'], optimized_hp['cnn_kernels_1']),
    'CNN Kernels L2': (baseline_hp['cnn_kernels_2'], optimized_hp['cnn_kernels_2']),
    'CNN Kernel Size L1': (baseline_hp['cnn_kernel_size_1'], optimized_hp['cnn_kernel_size_1']),
    'CNN Kernel Size L2': (baseline_hp['cnn_kernel_size_2'], optimized_hp.get('cnn_kernel_size_2', 3)),
    'CNN Dense': (baseline_hp['cnn_dense'], optimized_hp['cnn_dense']),
    'CNN Dropout': (baseline_hp['cnn_dropout'], optimized_hp['cnn_dropout']),
    'LSTM Hidden': (baseline_hp['lstm_hidden_size'], optimized_hp['lstm_hidden_size']),
    'LSTM Layers': (baseline_hp['lstm_layers'], optimized_hp['lstm_layers']),
    'LSTM Dense': (baseline_hp['lstm_dense'], optimized_hp['lstm_dense']),
    'Batch Size': (baseline_hp['batch_size'], optimized_hp['batch_size']),
}

# Plot 1: Hyperparameter Comparison (Bar Chart)
ax1 = plt.subplot(2, 3, 1)
metrics = list(comparison_metrics.keys())
baseline_vals = [comparison_metrics[m][0] for m in metrics]
optimized_vals = [comparison_metrics[m][1] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

bars1 = ax1.barh(x - width/2, baseline_vals, width, label='Baseline', alpha=0.8, color='steelblue')
bars2 = ax1.barh(x + width/2, optimized_vals, width, label='Optimized', alpha=0.8, color='coral')

ax1.set_xlabel('Parameter Value', fontsize=11, fontweight='bold')
ax1.set_ylabel('Hyperparameter', fontsize=11, fontweight='bold')
ax1.set_title('Hyperparameter Comparison', fontsize=13, fontweight='bold')
ax1.set_yticks(x)
ax1.set_yticklabels(metrics, fontsize=9)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Total Parameters Comparison
ax2 = plt.subplot(2, 3, 2)
models = ['Baseline', 'Optimized']
total_params = [
    baseline_df['total_params'].sum(),
    optimized_df['total_params'].sum()
]

bars = ax2.bar(models, total_params, color=['steelblue', 'coral'], alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('Number of Parameters', fontsize=11, fontweight='bold')
ax2.set_title('Total Model Parameters', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 3: Layer-wise Parameter Distribution
ax3 = plt.subplot(2, 3, 3)

# Group parameters by layer type
def group_by_type(df):
    type_params = {}
    for _, row in df.iterrows():
        layer_type = row['type']
        if layer_type not in type_params:
            type_params[layer_type] = 0
        type_params[layer_type] += row['total_params']
    return type_params

baseline_type_params = group_by_type(baseline_df)
optimized_type_params = group_by_type(optimized_df)

# Get all unique layer types
all_types = sorted(set(list(baseline_type_params.keys()) + list(optimized_type_params.keys())))

baseline_by_type = [baseline_type_params.get(t, 0) for t in all_types]
optimized_by_type = [optimized_type_params.get(t, 0) for t in all_types]

x_types = np.arange(len(all_types))
width = 0.35

ax3.bar(x_types - width/2, baseline_by_type, width, label='Baseline', alpha=0.8, color='steelblue')
ax3.bar(x_types + width/2, optimized_by_type, width, label='Optimized', alpha=0.8, color='coral')

ax3.set_xlabel('Layer Type', fontsize=11, fontweight='bold')
ax3.set_ylabel('Parameters', fontsize=11, fontweight='bold')
ax3.set_title('Parameters by Layer Type', fontsize=13, fontweight='bold')
ax3.set_xticks(x_types)
ax3.set_xticklabels(all_types, rotation=45, ha='right', fontsize=9)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Learning Rate and Optimizer
ax4 = plt.subplot(2, 3, 4)
ax4.axis('off')

comparison_text = f"""
TRAINING CONFIGURATION COMPARISON

Baseline (Unoptimized):
  • Optimizer: {baseline_hp['optimizer'].upper()}
  • Learning Rate: {baseline_hp['learning_rate']:.2e}
  • Batch Size: {baseline_hp['batch_size']}
  • Total Parameters: {baseline_df['total_params'].sum():,}

Optimized:
  • Optimizer: {optimized_hp['optimizer'].upper()}
  • Learning Rate: {optimized_hp['learning_rate']:.2e}
  • Batch Size: {optimized_hp['batch_size']}
  • Total Parameters: {optimized_df['total_params'].sum():,}

Parameter Reduction/Increase:
  • Difference: {optimized_df['total_params'].sum() - baseline_df['total_params'].sum():+,}
  • Ratio: {optimized_df['total_params'].sum() / baseline_df['total_params'].sum():.2f}x
"""

ax4.text(0.1, 0.9, comparison_text, transform=ax4.transAxes,
         fontsize=10, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# Plot 5: Architectural Differences Summary
ax5 = plt.subplot(2, 3, 5)
ax5.axis('off')

diff_text = "KEY ARCHITECTURAL DIFFERENCES:\n\n"
for metric, (base_val, opt_val) in comparison_metrics.items():
    if base_val != opt_val:
        change = opt_val - base_val
        pct_change = ((opt_val - base_val) / base_val * 100) if base_val != 0 else 0
        diff_text += f"• {metric}:\n"
        diff_text += f"  {base_val} → {opt_val} ({change:+.1f}, {pct_change:+.1f}%)\n\n"

if diff_text == "KEY ARCHITECTURAL DIFFERENCES:\n\n":
    diff_text += "No architectural differences found.\nBoth models use identical structures."

ax5.text(0.05, 0.95, diff_text, transform=ax5.transAxes,
         fontsize=9, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

# Plot 6: Performance Comparison
ax6 = plt.subplot(2, 3, 6)

metrics_names = ['Test Accuracy', 'Best Val Acc', 'Best Val Loss']
baseline_metrics = [
    baseline_results['test_accuracy'],
    baseline_results['best_val_accuracy'],
    baseline_results['best_val_loss']
]
optimized_metrics = [
    acc,  # test accuracy from optimized model
    max(history['val_accs']),
    min(history['val_losses'])
]

x_perf = np.arange(len(metrics_names))
width = 0.35

# Normalize for visualization (loss is inverted)
baseline_display = baseline_metrics.copy()
optimized_display = optimized_metrics.copy()
baseline_display[2] = 1 - baseline_display[2]  # Invert loss for display
optimized_display[2] = 1 - optimized_display[2]

bars1 = ax6.bar(x_perf - width/2, baseline_display, width, label='Baseline', alpha=0.8, color='steelblue')
bars2 = ax6.bar(x_perf + width/2, optimized_display, width, label='Optimized', alpha=0.8, color='coral')

ax6.set_ylabel('Score', fontsize=11, fontweight='bold')
ax6.set_title('Performance Comparison', fontsize=13, fontweight='bold')
ax6.set_xticks(x_perf)
ax6.set_xticklabels(metrics_names, fontsize=9)
ax6.legend()
ax6.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for i, bar in enumerate(bars):
        height = bar.get_height()
        actual_val = baseline_metrics[i] if bars == bars1 else optimized_metrics[i]
        ax6.text(bar.get_x() + bar.get_width()/2., height,
                f'{actual_val:.4f}',
                ha='center', va='bottom', fontsize=8)

plt.suptitle('Comprehensive Architecture & Performance Comparison', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

print("\n✓ Visual comparison complete!")

In [ ]:
# Summary Report: Architecture Analysis
print("\n" + "="*70)
print("ARCHITECTURE ANALYSIS SUMMARY")
print("="*70)

print("\n1. COMPUTATIONAL GRAPH VISUALIZATIONS:")
print("   ✓ Baseline model graph: baseline_model_graph.png")
print("   ✓ Optimized model graph: optimized_model_graph.png")
print("   → Compare data flow, operation sequences, and layer connections")

print("\n2. STRUCTURAL DIFFERENCES:")
baseline_total = baseline_df['total_params'].sum()
optimized_total = optimized_df['total_params'].sum()
param_diff = optimized_total - baseline_total
param_ratio = optimized_total / baseline_total

print(f"   • Baseline Model: {baseline_total:,} parameters")
print(f"   • Optimized Model: {optimized_total:,} parameters")
print(f"   • Difference: {param_diff:+,} ({(param_ratio-1)*100:+.2f}%)")

print("\n3. LAYER COMPOSITION:")
print(f"   Baseline: {len(baseline_df)} layers")
print(f"   Optimized: {len(optimized_df)} layers")

print("\n4. OPTIMIZATION IMPACT:")
perf_improvement = (acc - baseline_results['test_accuracy']) * 100
print(f"   • Test Accuracy Improvement: {perf_improvement:+.2f}%")
print(f"   • Baseline: {baseline_results['test_accuracy']:.4f}")
print(f"   • Optimized: {acc:.4f}")

print("\n5. KEY ARCHITECTURAL CHANGES:")
for metric, (base_val, opt_val) in comparison_metrics.items():
    if base_val != opt_val:
        print(f"   • {metric}: {base_val} → {opt_val}")

print("\n6. TRAINING EFFICIENCY:")
print(f"   • Baseline epochs: {baseline_results['epochs_trained']}")
print(f"   • Optimized epochs: {len(history['train_losses'])}")
print(f"   • Baseline optimizer: {baseline_hp['optimizer']}")
print(f"   • Optimized optimizer: {optimized_hp['optimizer']}")

print("\n" + "="*70)
print("✓ Complete architecture analysis finished!")
print("="*70)

#### **Model Internals Visualization: Feature Maps & Hidden States**

Deep dive into learned representations to understand model behavior:

---

### **Visualization Goals**

Compare internal representations between baseline (unoptimized) and optimized models:

1. **CNN Feature Maps**: Visualize learned spatial-temporal patterns at each convolutional layer
2. **LSTM Hidden States**: Track temporal evolution of hidden state activations across sequence

---

### **Why Visualize Internal Representations?**

**CNN Feature Maps:**
- Reveal what spatial-temporal patterns the model has learned to detect
- Show differences in feature complexity between layers
- Compare feature learning effectiveness between optimized vs unoptimized models
- Identify if optimization leads to more discriminative features

**LSTM Hidden States:**
- Understand temporal pattern evolution across the EEG sequence
- Visualize how the model tracks information over time
- Compare temporal modeling between baseline and optimized architectures
- Identify key timesteps where classification decisions are made

---

### **Interpretation Guide**

**Feature Map Patterns:**
- Bright/dark regions indicate strong activations (detected patterns)
- Layer 1: Low-level features (edges, simple patterns)
- Layer 2: Higher-level features (complex spatial-temporal combinations)
- Optimized models should show clearer, more structured patterns

**Hidden State Dynamics:**
- Each row represents one LSTM layer's hidden state
- Color intensity shows activation magnitude
- Temporal evolution (left to right) shows information flow
- Final states (rightmost) influence classification decision

---

### **Sample Selection**

For visualization, we'll use:
- Representative samples from each ADHD class
- Focus on correctly classified examples to understand successful pattern detection
- Compare activations between baseline and optimized models

In [ ]:
# Helper class to capture intermediate activations
class FeatureExtractor:
    """Extract CNN feature maps and LSTM hidden states during forward pass."""
    
    def __init__(self, model):
        self.model = model
        self.cnn_features = {}
        self.lstm_states = None
        self.hooks = []
        self._register_hooks()
    
    def _register_hooks(self):
        """Register forward hooks to capture activations."""
        
        # Hook for first conv layer
        def hook_conv1(module, input, output):
            self.cnn_features['conv1'] = output.detach()
        
        # Hook for second conv layer
        def hook_conv2(module, input, output):
            self.cnn_features['conv2'] = output.detach()
        
        # Hook for LSTM
        def hook_lstm(module, input, output):
            # output is (lstm_out, (h_n, c_n))
            lstm_out, (h_n, c_n) = output
            self.lstm_states = {
                'output': lstm_out.detach(),  # [B, seq_len, hidden_size]
                'hidden': h_n.detach(),       # [num_layers, B, hidden_size]
                'cell': c_n.detach()          # [num_layers, B, hidden_size]
            }
        
        # Register hooks
        self.hooks.append(self.model.conv1.register_forward_hook(hook_conv1))
        self.hooks.append(self.model.conv2.register_forward_hook(hook_conv2))
        self.hooks.append(self.model.lstm.register_forward_hook(hook_lstm))
    
    def extract(self, x_eeg, x_band):
        """Forward pass and return features."""
        self.cnn_features = {}
        self.lstm_states = None
        
        with torch.no_grad():
            output = self.model(x_eeg, x_band)
        
        return output, self.cnn_features, self.lstm_states
    
    def remove_hooks(self):
        """Clean up hooks."""
        for hook in self.hooks:
            hook.remove()

print("✓ FeatureExtractor class defined successfully")

In [ ]:
# Select sample for visualization
print("Selecting representative samples from test set...")

# Get one sample from each class
baseline_model.eval()
model.eval()

sample_indices = {}
with torch.no_grad():
    for idx, (xb_eeg, xb_band, yb) in enumerate(baseline_test_loader):
        for i in range(len(yb)):
            label = int(yb[i].item())
            if label not in sample_indices:
                sample_indices[label] = {
                    'x_eeg': xb_eeg[i:i+1].to(device),
                    'x_band': xb_band[i:i+1].to(device),
                    'label': label
                }
            if len(sample_indices) == 4:  # Assuming 4 classes
                break
        if len(sample_indices) == 4:
            break

print(f"✓ Selected {len(sample_indices)} samples (one per class)")
print(f"Classes represented: {list(sample_indices.keys())}")

In [ ]:
# Extract features from both models for visualization
print("\nExtracting CNN feature maps and LSTM hidden states...")

# Choose one sample for detailed visualization (class 0)
sample = sample_indices[0]
x_eeg_sample = sample['x_eeg']
x_band_sample = sample['x_band']

# Extract from baseline model
baseline_extractor = FeatureExtractor(baseline_model)
baseline_output, baseline_cnn_feats, baseline_lstm_states = baseline_extractor.extract(x_eeg_sample, x_band_sample)

# Extract from optimized model
optimized_extractor = FeatureExtractor(model)
optimized_output, optimized_cnn_feats, optimized_lstm_states = optimized_extractor.extract(x_eeg_sample, x_band_sample)

print("✓ Feature extraction complete")
print(f"\nBaseline Model:")
print(f"  Conv1 output shape: {baseline_cnn_feats['conv1'].shape}")
print(f"  Conv2 output shape: {baseline_cnn_feats['conv2'].shape}")
print(f"  LSTM output shape: {baseline_lstm_states['output'].shape}")
print(f"  LSTM hidden states: {baseline_lstm_states['hidden'].shape}")

print(f"\nOptimized Model:")
print(f"  Conv1 output shape: {optimized_cnn_feats['conv1'].shape}")
print(f"  Conv2 output shape: {optimized_cnn_feats['conv2'].shape}")
print(f"  LSTM output shape: {optimized_lstm_states['output'].shape}")
print(f"  LSTM hidden states: {optimized_lstm_states['hidden'].shape}")

# Predictions
baseline_pred = baseline_output.argmax(1).item()
optimized_pred = optimized_output.argmax(1).item()
true_label = sample['label']

print(f"\nPredictions for sample (True label: {true_label}):")
print(f"  Baseline: {baseline_pred}")
print(f"  Optimized: {optimized_pred}")

In [ ]:
# Visualize CNN Feature Maps - Comparison
import matplotlib.pyplot as plt
import numpy as np

def visualize_cnn_features(baseline_feats, optimized_feats, num_filters=8):
    """Visualize and compare CNN feature maps from both models."""
    
    fig = plt.figure(figsize=(20, 10))
    
    # Baseline Conv1
    conv1_baseline = baseline_feats['conv1'][0].cpu().numpy()  # [C, H, W]
    num_show = min(num_filters, conv1_baseline.shape[0])
    
    for i in range(num_show):
        ax = plt.subplot(4, num_show, i + 1)
        im = ax.imshow(conv1_baseline[i], cmap='viridis', aspect='auto')
        ax.set_title(f'Filter {i+1}', fontsize=10, fontweight='bold')
        ax.axis('off')
        if i == 0:
            ax.set_ylabel('Baseline\nConv1', fontsize=11, fontweight='bold', rotation=0, 
                         labelpad=40, va='center')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # Optimized Conv1
    conv1_optimized = optimized_feats['conv1'][0].cpu().numpy()
    num_show_opt = min(num_filters, conv1_optimized.shape[0])
    
    for i in range(num_show_opt):
        ax = plt.subplot(4, num_show, num_show + i + 1)
        im = ax.imshow(conv1_optimized[i], cmap='viridis', aspect='auto')
        ax.set_title(f'Filter {i+1}', fontsize=10, fontweight='bold')
        ax.axis('off')
        if i == 0:
            ax.set_ylabel('Optimized\nConv1', fontsize=11, fontweight='bold', rotation=0, 
                         labelpad=40, va='center')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # Baseline Conv2
    conv2_baseline = baseline_feats['conv2'][0].cpu().numpy()
    num_show = min(num_filters, conv2_baseline.shape[0])
    
    for i in range(num_show):
        ax = plt.subplot(4, num_show, 2*num_show + i + 1)
        im = ax.imshow(conv2_baseline[i], cmap='plasma', aspect='auto')
        ax.set_title(f'Filter {i+1}', fontsize=10, fontweight='bold')
        ax.axis('off')
        if i == 0:
            ax.set_ylabel('Baseline\nConv2', fontsize=11, fontweight='bold', rotation=0, 
                         labelpad=40, va='center')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # Optimized Conv2
    conv2_optimized = optimized_feats['conv2'][0].cpu().numpy()
    num_show_opt = min(num_filters, conv2_optimized.shape[0])
    
    for i in range(num_show_opt):
        ax = plt.subplot(4, num_show, 3*num_show + i + 1)
        im = ax.imshow(conv2_optimized[i], cmap='plasma', aspect='auto')
        ax.set_title(f'Filter {i+1}', fontsize=10, fontweight='bold')
        ax.axis('off')
        if i == 0:
            ax.set_ylabel('Optimized\nConv2', fontsize=11, fontweight='bold', rotation=0, 
                         labelpad=40, va='center')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    plt.suptitle('CNN Feature Maps Comparison: Baseline vs Optimized', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout(rect=[0, 0, 1, 0.99])
    plt.show()

print("Visualizing CNN feature maps...")
visualize_cnn_features(baseline_cnn_feats, optimized_cnn_feats, num_filters=8)
print("✓ CNN feature maps visualization complete")

In [ ]:
# Visualize LSTM Hidden State Activations Over Time
def visualize_lstm_states(baseline_states, optimized_states):
    """Visualize and compare LSTM hidden state evolution over time."""
    
    # Extract hidden states [num_layers, batch, hidden_size]
    # We'll visualize the evolution of LSTM output over sequence [batch, seq_len, hidden_size]
    baseline_lstm_out = baseline_states['output'][0].cpu().numpy()  # [seq_len, hidden_size]
    optimized_lstm_out = optimized_states['output'][0].cpu().numpy()
    
    # Also get final hidden states for each layer
    baseline_hidden = baseline_states['hidden'][:, 0, :].cpu().numpy()  # [num_layers, hidden_size]
    optimized_hidden = optimized_states['hidden'][:, 0, :].cpu().numpy()
    
    fig = plt.figure(figsize=(20, 12))
    
    # Plot 1: Baseline LSTM output evolution over time
    ax1 = plt.subplot(3, 2, 1)
    im1 = ax1.imshow(baseline_lstm_out.T, cmap='RdBu_r', aspect='auto', interpolation='nearest')
    ax1.set_xlabel('Time Step', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Hidden Unit', fontsize=12, fontweight='bold')
    ax1.set_title('Baseline: LSTM Output Over Time', fontsize=14, fontweight='bold')
    plt.colorbar(im1, ax=ax1, label='Activation')
    
    # Plot 2: Optimized LSTM output evolution over time
    ax2 = plt.subplot(3, 2, 2)
    im2 = ax2.imshow(optimized_lstm_out.T, cmap='RdBu_r', aspect='auto', interpolation='nearest')
    ax2.set_xlabel('Time Step', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Hidden Unit', fontsize=12, fontweight='bold')
    ax2.set_title('Optimized: LSTM Output Over Time', fontsize=14, fontweight='bold')
    plt.colorbar(im2, ax=ax2, label='Activation')
    
    # Plot 3: Baseline layer-wise final hidden states
    ax3 = plt.subplot(3, 2, 3)
    im3 = ax3.imshow(baseline_hidden, cmap='viridis', aspect='auto', interpolation='nearest')
    ax3.set_xlabel('Hidden Unit', fontsize=12, fontweight='bold')
    ax3.set_ylabel('LSTM Layer', fontsize=12, fontweight='bold')
    ax3.set_title('Baseline: Final Hidden States (All Layers)', fontsize=14, fontweight='bold')
    ax3.set_yticks(range(baseline_hidden.shape[0]))
    ax3.set_yticklabels([f'Layer {i+1}' for i in range(baseline_hidden.shape[0])])
    plt.colorbar(im3, ax=ax3, label='Activation')
    
    # Plot 4: Optimized layer-wise final hidden states
    ax4 = plt.subplot(3, 2, 4)
    im4 = ax4.imshow(optimized_hidden, cmap='viridis', aspect='auto', interpolation='nearest')
    ax4.set_xlabel('Hidden Unit', fontsize=12, fontweight='bold')
    ax4.set_ylabel('LSTM Layer', fontsize=12, fontweight='bold')
    ax4.set_title('Optimized: Final Hidden States (All Layers)', fontsize=14, fontweight='bold')
    ax4.set_yticks(range(optimized_hidden.shape[0]))
    ax4.set_yticklabels([f'Layer {i+1}' for i in range(optimized_hidden.shape[0])])
    plt.colorbar(im4, ax=ax4, label='Activation')
    
    # Plot 5: Temporal activation statistics (baseline)
    ax5 = plt.subplot(3, 2, 5)
    timesteps = np.arange(baseline_lstm_out.shape[0])
    mean_activation = np.mean(np.abs(baseline_lstm_out), axis=1)
    std_activation = np.std(baseline_lstm_out, axis=1)
    ax5.plot(timesteps, mean_activation, 'b-', linewidth=2, label='Mean |Activation|')
    ax5.fill_between(timesteps, mean_activation - std_activation, 
                     mean_activation + std_activation, alpha=0.3, color='blue', label='±1 Std Dev')
    ax5.set_xlabel('Time Step', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Activation Magnitude', fontsize=12, fontweight='bold')
    ax5.set_title('Baseline: Temporal Activation Statistics', fontsize=14, fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Temporal activation statistics (optimized)
    ax6 = plt.subplot(3, 2, 6)
    timesteps_opt = np.arange(optimized_lstm_out.shape[0])
    mean_activation_opt = np.mean(np.abs(optimized_lstm_out), axis=1)
    std_activation_opt = np.std(optimized_lstm_out, axis=1)
    ax6.plot(timesteps_opt, mean_activation_opt, 'r-', linewidth=2, label='Mean |Activation|')
    ax6.fill_between(timesteps_opt, mean_activation_opt - std_activation_opt, 
                     mean_activation_opt + std_activation_opt, alpha=0.3, color='red', label='±1 Std Dev')
    ax6.set_xlabel('Time Step', fontsize=12, fontweight='bold')
    ax6.set_ylabel('Activation Magnitude', fontsize=12, fontweight='bold')
    ax6.set_title('Optimized: Temporal Activation Statistics', fontsize=14, fontweight='bold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.suptitle('LSTM Hidden State Activations: Baseline vs Optimized', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout(rect=[0, 0, 1, 0.99])
    plt.show()
    
    # Print statistics
    print("\n" + "="*70)
    print("LSTM ACTIVATION STATISTICS")
    print("="*70)
    print(f"\nBaseline Model:")
    print(f"  LSTM Layers: {baseline_hidden.shape[0]}")
    print(f"  Hidden Size: {baseline_hidden.shape[1]}")
    print(f"  Sequence Length: {baseline_lstm_out.shape[0]}")
    print(f"  Mean Activation: {np.mean(np.abs(baseline_lstm_out)):.4f}")
    print(f"  Std Activation: {np.std(baseline_lstm_out):.4f}")
    
    print(f"\nOptimized Model:")
    print(f"  LSTM Layers: {optimized_hidden.shape[0]}")
    print(f"  Hidden Size: {optimized_hidden.shape[1]}")
    print(f"  Sequence Length: {optimized_lstm_out.shape[0]}")
    print(f"  Mean Activation: {np.mean(np.abs(optimized_lstm_out)):.4f}")
    print(f"  Std Activation: {np.std(optimized_lstm_out):.4f}")
    print("="*70)

print("\nVisualizing LSTM hidden state activations...")
visualize_lstm_states(baseline_lstm_states, optimized_lstm_states)
print("✓ LSTM hidden state visualization complete")

In [ ]:
# Comparative Analysis: Feature Activation Patterns
def comparative_analysis(baseline_cnn, optimized_cnn, baseline_lstm, optimized_lstm):
    """Quantitative comparison of learned representations."""
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Extract data
    baseline_conv1 = baseline_cnn['conv1'][0].cpu().numpy()
    optimized_conv1 = optimized_cnn['conv1'][0].cpu().numpy()
    baseline_conv2 = baseline_cnn['conv2'][0].cpu().numpy()
    optimized_conv2 = optimized_cnn['conv2'][0].cpu().numpy()
    
    # Plot 1: Filter activation magnitude comparison (Conv1)
    ax1 = axes[0, 0]
    baseline_conv1_mag = np.mean(np.abs(baseline_conv1), axis=(1, 2))
    optimized_conv1_mag = np.mean(np.abs(optimized_conv1), axis=(1, 2))
    
    x = np.arange(len(baseline_conv1_mag))
    width = 0.35
    ax1.bar(x - width/2, baseline_conv1_mag, width, label='Baseline', alpha=0.8, color='steelblue')
    ax1.bar(x + width/2, optimized_conv1_mag[:len(x)], width, label='Optimized', alpha=0.8, color='coral')
    ax1.set_xlabel('Filter Index', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Mean Absolute Activation', fontsize=12, fontweight='bold')
    ax1.set_title('Conv1: Filter Activation Magnitudes', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Filter activation magnitude comparison (Conv2)
    ax2 = axes[0, 1]
    baseline_conv2_mag = np.mean(np.abs(baseline_conv2), axis=(1, 2))
    optimized_conv2_mag = np.mean(np.abs(optimized_conv2), axis=(1, 2))
    
    x2 = np.arange(len(baseline_conv2_mag))
    ax2.bar(x2 - width/2, baseline_conv2_mag, width, label='Baseline', alpha=0.8, color='steelblue')
    ax2.bar(x2 + width/2, optimized_conv2_mag[:len(x2)], width, label='Optimized', alpha=0.8, color='coral')
    ax2.set_xlabel('Filter Index', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Mean Absolute Activation', fontsize=12, fontweight='bold')
    ax2.set_title('Conv2: Filter Activation Magnitudes', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Plot 3: LSTM activation distribution
    ax3 = axes[1, 0]
    baseline_lstm_out = baseline_lstm['output'][0].cpu().numpy().flatten()
    optimized_lstm_out = optimized_lstm['output'][0].cpu().numpy().flatten()
    
    ax3.hist(baseline_lstm_out, bins=50, alpha=0.6, label='Baseline', color='steelblue', density=True)
    ax3.hist(optimized_lstm_out, bins=50, alpha=0.6, label='Optimized', color='coral', density=True)
    ax3.set_xlabel('Activation Value', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Density', fontsize=12, fontweight='bold')
    ax3.set_title('LSTM: Activation Distribution', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Plot 4: Activation sparsity (percentage of near-zero activations)
    ax4 = axes[1, 1]
    
    def compute_sparsity(features_dict, threshold=0.1):
        """Compute percentage of activations below threshold."""
        sparsity = {}
        for key, val in features_dict.items():
            if 'conv' in key:
                data = val[0].cpu().numpy().flatten()
                sparsity[key] = (np.abs(data) < threshold).sum() / len(data) * 100
        return sparsity
    
    baseline_sparsity = compute_sparsity(baseline_cnn)
    optimized_sparsity = compute_sparsity(optimized_cnn)
    
    layers = list(baseline_sparsity.keys())
    baseline_vals = [baseline_sparsity[k] for k in layers]
    optimized_vals = [optimized_sparsity[k] for k in layers]
    
    x4 = np.arange(len(layers))
    ax4.bar(x4 - width/2, baseline_vals, width, label='Baseline', alpha=0.8, color='steelblue')
    ax4.bar(x4 + width/2, optimized_vals, width, label='Optimized', alpha=0.8, color='coral')
    ax4.set_xlabel('Layer', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Sparsity (%)', fontsize=12, fontweight='bold')
    ax4.set_title('CNN: Activation Sparsity (|activation| < 0.1)', fontsize=14, fontweight='bold')
    ax4.set_xticks(x4)
    ax4.set_xticklabels(layers)
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('Comparative Analysis: Feature Activation Patterns', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout(rect=[0, 0, 1, 0.99])
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*70)
    print("COMPARATIVE FEATURE ANALYSIS SUMMARY")
    print("="*70)
    
    print("\nCNN Feature Statistics:")
    print(f"  Baseline Conv1 - Mean: {np.mean(baseline_conv1_mag):.4f}, Std: {np.std(baseline_conv1_mag):.4f}")
    print(f"  Optimized Conv1 - Mean: {np.mean(optimized_conv1_mag):.4f}, Std: {np.std(optimized_conv1_mag):.4f}")
    print(f"  Baseline Conv2 - Mean: {np.mean(baseline_conv2_mag):.4f}, Std: {np.std(baseline_conv2_mag):.4f}")
    print(f"  Optimized Conv2 - Mean: {np.mean(optimized_conv2_mag):.4f}, Std: {np.std(optimized_conv2_mag):.4f}")
    
    print("\nLSTM Activation Statistics:")
    print(f"  Baseline - Mean: {np.mean(baseline_lstm_out):.4f}, Std: {np.std(baseline_lstm_out):.4f}")
    print(f"  Optimized - Mean: {np.mean(optimized_lstm_out):.4f}, Std: {np.std(optimized_lstm_out):.4f}")
    
    print("\nSparsity Analysis:")
    for layer in layers:
        print(f"  {layer} - Baseline: {baseline_sparsity[layer]:.2f}%, Optimized: {optimized_sparsity[layer]:.2f}%")
    
    print("="*70)

print("\nPerforming comparative analysis...")
comparative_analysis(baseline_cnn_feats, optimized_cnn_feats, 
                    baseline_lstm_states, optimized_lstm_states)
print("✓ Comparative analysis complete")

In [ ]:
# Cleanup: Remove hooks
baseline_extractor.remove_hooks()
optimized_extractor.remove_hooks()

print("\n" + "="*70)
print("VISUALIZATION SUMMARY")
print("="*70)
print("\nKey Insights from Internal Representations:")
print("\n1. CNN Feature Maps:")
print("   - Conv1: Captures low-level spatial-temporal patterns")
print("   - Conv2: Extracts higher-level feature combinations")
print("   - Compare activation magnitudes and patterns between models")
print("\n2. LSTM Hidden States:")
print("   - Temporal evolution shows information flow across sequence")
print("   - Final hidden states encode compressed temporal information")
print("   - Layer-wise activations reveal hierarchical temporal processing")
print("\n3. Comparative Analysis:")
print("   - Filter activation magnitudes show feature importance")
print("   - Activation distributions reveal learning characteristics")
print("   - Sparsity indicates selective feature detection")
print("\n4. Optimization Impact:")
print("   - Optimized model may show more structured/selective activations")
print("   - Better hyperparameters can lead to clearer feature separation")
print("   - LSTM architecture changes affect temporal modeling capacity")
print("="*70)
print("\n✓ All visualizations complete!")

---

#### **Training Dynamics & Performance Comparison**

Comprehensive side-by-side comparison of training behavior and final performance between baseline (unoptimized) and optimized models:

---

### **Comparison Goals**

**Training Curves Analysis:**
- Compare convergence speed and stability
- Identify overfitting patterns and generalization gaps
- Assess impact of hyperparameter optimization on learning dynamics
- Evaluate early stopping effectiveness

**Confusion Matrix Analysis:**
- Compare classification performance across all classes
- Identify class-specific improvements
- Understand misclassification patterns
- Quantify optimization impact on per-class accuracy

---

### **Key Metrics to Compare**

**Training Dynamics:**
- Convergence rate (epochs to best performance)
- Training vs validation gap (overfitting measure)
- Final training and validation accuracy/loss
- Learning curve smoothness and stability

**Classification Performance:**
- Overall accuracy improvement
- Per-class precision, recall, and F1-score
- Confusion matrix diagonal strength (correct predictions)
- Reduction in misclassifications

---

### **Interpretation Guide**

**Training Curves:**
- **Faster convergence** → Better hyperparameters for learning
- **Smaller train-val gap** → Better regularization and generalization
- **Smoother curves** → More stable training process
- **Higher final accuracy** → Better overall optimization

**Confusion Matrices:**
- **Brighter diagonal** → More correct predictions
- **Darker off-diagonal** → Fewer misclassifications
- **Class balance** → Even performance across categories
- **Comparison highlights** → Classes that benefited most from optimization

In [ ]:
# Side-by-Side Training Curves Comparison
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(20, 10))

# Extract training histories
baseline_train_acc = baseline_history['train_accs']
baseline_val_acc = baseline_history['val_accs']
baseline_train_loss = baseline_history['train_losses']
baseline_val_loss = baseline_history['val_losses']

optimized_train_acc = history['train_accs']
optimized_val_acc = history['val_accs']
optimized_train_loss = history['train_losses']
optimized_val_loss = history['val_losses']

# Create epochs arrays
baseline_epochs = np.arange(1, len(baseline_train_acc) + 1)
optimized_epochs = np.arange(1, len(optimized_train_acc) + 1)

# Plot 1: Baseline Accuracy
ax1 = plt.subplot(2, 3, 1)
ax1.plot(baseline_epochs, baseline_train_acc, 'b-', linewidth=2, label='Training', alpha=0.8)
ax1.plot(baseline_epochs, baseline_val_acc, 'r-', linewidth=2, label='Validation', alpha=0.8)
ax1.axhline(y=max(baseline_val_acc), color='green', linestyle='--', alpha=0.5, 
            label=f'Best Val: {max(baseline_val_acc):.4f}')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Baseline: Accuracy Curves', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Plot 2: Optimized Accuracy
ax2 = plt.subplot(2, 3, 2)
ax2.plot(optimized_epochs, optimized_train_acc, 'b-', linewidth=2, label='Training', alpha=0.8)
ax2.plot(optimized_epochs, optimized_val_acc, 'r-', linewidth=2, label='Validation', alpha=0.8)
ax2.axhline(y=max(optimized_val_acc), color='green', linestyle='--', alpha=0.5,
            label=f'Best Val: {max(optimized_val_acc):.4f}')
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax2.set_title('Optimized: Accuracy Curves', fontsize=14, fontweight='bold')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

# Plot 3: Overlay Comparison - Validation Accuracy
ax3 = plt.subplot(2, 3, 3)
ax3.plot(baseline_epochs, baseline_val_acc, 'steelblue', linewidth=2.5, 
         label=f'Baseline (Best: {max(baseline_val_acc):.4f})', alpha=0.8)
ax3.plot(optimized_epochs, optimized_val_acc, 'coral', linewidth=2.5,
         label=f'Optimized (Best: {max(optimized_val_acc):.4f})', alpha=0.8)
ax3.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax3.set_ylabel('Validation Accuracy', fontsize=12, fontweight='bold')
ax3.set_title('Validation Accuracy Comparison', fontsize=14, fontweight='bold')
ax3.legend(loc='lower right')
ax3.grid(True, alpha=0.3)
ax3.set_ylim([0, 1])

# Plot 4: Baseline Loss
ax4 = plt.subplot(2, 3, 4)
ax4.plot(baseline_epochs, baseline_train_loss, 'b-', linewidth=2, label='Training', alpha=0.8)
ax4.plot(baseline_epochs, baseline_val_loss, 'r-', linewidth=2, label='Validation', alpha=0.8)
ax4.axhline(y=min(baseline_val_loss), color='green', linestyle='--', alpha=0.5,
            label=f'Best Val: {min(baseline_val_loss):.4f}')
ax4.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax4.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax4.set_title('Baseline: Loss Curves', fontsize=14, fontweight='bold')
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)

# Plot 5: Optimized Loss
ax5 = plt.subplot(2, 3, 5)
ax5.plot(optimized_epochs, optimized_train_loss, 'b-', linewidth=2, label='Training', alpha=0.8)
ax5.plot(optimized_epochs, optimized_val_loss, 'r-', linewidth=2, label='Validation', alpha=0.8)
ax5.axhline(y=min(optimized_val_loss), color='green', linestyle='--', alpha=0.5,
            label=f'Best Val: {min(optimized_val_loss):.4f}')
ax5.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax5.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax5.set_title('Optimized: Loss Curves', fontsize=14, fontweight='bold')
ax5.legend(loc='upper right')
ax5.grid(True, alpha=0.3)

# Plot 6: Overlay Comparison - Validation Loss
ax6 = plt.subplot(2, 3, 6)
ax6.plot(baseline_epochs, baseline_val_loss, 'steelblue', linewidth=2.5,
         label=f'Baseline (Best: {min(baseline_val_loss):.4f})', alpha=0.8)
ax6.plot(optimized_epochs, optimized_val_loss, 'coral', linewidth=2.5,
         label=f'Optimized (Best: {min(optimized_val_loss):.4f})', alpha=0.8)
ax6.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax6.set_ylabel('Validation Loss', fontsize=12, fontweight='bold')
ax6.set_title('Validation Loss Comparison', fontsize=14, fontweight='bold')
ax6.legend(loc='upper right')
ax6.grid(True, alpha=0.3)

plt.suptitle('Training Dynamics: Baseline vs Optimized', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

print("✓ Training curves comparison complete")

In [ ]:
# Generate confusion matrices for both models
print("Generating confusion matrices for comparison...")

# Baseline model confusion matrix (already computed during baseline evaluation)
baseline_cm = conf_matrix  # From baseline evaluation

# Optimized model confusion matrix
optimized_model = model  # The optimized model
optimized_model.eval()

all_preds_opt = []
all_labels_opt = []

with torch.no_grad():
    for xb_eeg, xb_band, yb in test_loader:
        xb_eeg = xb_eeg.to(device)
        xb_band = xb_band.to(device)
        preds = optimized_model(xb_eeg, xb_band).argmax(1).cpu().numpy()
        all_preds_opt.append(preds)
        all_labels_opt.append(yb.numpy())

all_preds_opt = np.concatenate(all_preds_opt)
all_labels_opt = np.concatenate(all_labels_opt)

optimized_cm = confusion_matrix(all_labels_opt, all_preds_opt)
optimized_acc = accuracy_score(all_labels_opt, all_preds_opt)

print(f"✓ Baseline Test Accuracy: {test_accuracy:.4f}")
print(f"✓ Optimized Test Accuracy: {optimized_acc:.4f}")
print(f"✓ Improvement: {(optimized_acc - test_accuracy):.4f} ({(optimized_acc - test_accuracy)*100:.2f}%)")

In [ ]:
# Side-by-Side Confusion Matrix Comparison
import seaborn as sns

fig = plt.figure(figsize=(20, 8))

# Determine number of classes
n_classes = baseline_cm.shape[0]
class_labels = [f'Class {i}' for i in range(n_classes)]

# Plot 1: Baseline Confusion Matrix
ax1 = plt.subplot(1, 3, 1)
sns.heatmap(baseline_cm, annot=True, fmt='d', cmap='Blues', square=True, 
            cbar_kws={'label': 'Count'}, ax=ax1, 
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=0.5, linecolor='gray')
ax1.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
ax1.set_ylabel('True Label', fontsize=12, fontweight='bold')
ax1.set_title(f'Baseline Confusion Matrix\nAccuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)', 
              fontsize=14, fontweight='bold')

# Plot 2: Optimized Confusion Matrix
ax2 = plt.subplot(1, 3, 2)
sns.heatmap(optimized_cm, annot=True, fmt='d', cmap='Oranges', square=True,
            cbar_kws={'label': 'Count'}, ax=ax2,
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=0.5, linecolor='gray')
ax2.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
ax2.set_ylabel('True Label', fontsize=12, fontweight='bold')
ax2.set_title(f'Optimized Confusion Matrix\nAccuracy: {optimized_acc:.4f} ({optimized_acc*100:.2f}%)', 
              fontsize=14, fontweight='bold')

# Plot 3: Difference Heatmap (Optimized - Baseline)
ax3 = plt.subplot(1, 3, 3)
diff_cm = optimized_cm - baseline_cm

# Use diverging colormap centered at 0
vmax = max(abs(diff_cm.min()), abs(diff_cm.max()))
sns.heatmap(diff_cm, annot=True, fmt='d', cmap='RdBu_r', center=0, 
            square=True, cbar_kws={'label': 'Difference'},
            vmin=-vmax, vmax=vmax, ax=ax3,
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=0.5, linecolor='gray')
ax3.set_xlabel('Predicted Label', fontsize=12, fontweight='bold')
ax3.set_ylabel('True Label', fontsize=12, fontweight='bold')
improvement = (optimized_acc - test_accuracy) * 100
ax3.set_title(f'Improvement (Optimized - Baseline)\nΔ Accuracy: {improvement:+.2f}%', 
              fontsize=14, fontweight='bold')

plt.suptitle('Confusion Matrix Comparison: Baseline vs Optimized', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("✓ Confusion matrix comparison complete")

In [ ]:
# Per-Class Performance Comparison
from sklearn.metrics import classification_report

# Get detailed classification reports
baseline_report = classification_report(all_labels, all_preds, output_dict=True)
optimized_report = classification_report(all_labels_opt, all_preds_opt, output_dict=True)

# Extract per-class metrics
classes = sorted([k for k in baseline_report.keys() if k.isdigit()])
metrics = ['precision', 'recall', 'f1-score']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    baseline_vals = [baseline_report[c][metric] for c in classes]
    optimized_vals = [optimized_report[c][metric] for c in classes]
    
    x = np.arange(len(classes))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, baseline_vals, width, label='Baseline', 
                   alpha=0.8, color='steelblue', edgecolor='black', linewidth=1.5)
    bars2 = ax.bar(x + width/2, optimized_vals, width, label='Optimized',
                   alpha=0.8, color='coral', edgecolor='black', linewidth=1.5)
    
    ax.set_xlabel('Class', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric.replace('-', ' ').title(), fontsize=12, fontweight='bold')
    ax.set_title(f'Per-Class {metric.replace("-", " ").title()} Comparison', 
                 fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([f'Class {c}' for c in classes])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim([0, 1])
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=8)

plt.suptitle('Per-Class Performance Metrics: Baseline vs Optimized', 
             fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("✓ Per-class performance comparison complete")

In [ ]:
# Training Dynamics Summary Statistics
print("\n" + "="*70)
print("TRAINING DYNAMICS SUMMARY")
print("="*70)

print("\n" + "-"*70)
print("BASELINE MODEL (UNOPTIMIZED)")
print("-"*70)
print(f"Total Epochs Trained: {len(baseline_train_acc)}")
print(f"\nTraining Performance:")
print(f"  Final Training Accuracy: {baseline_train_acc[-1]:.4f}")
print(f"  Final Training Loss: {baseline_train_loss[-1]:.4f}")
print(f"\nValidation Performance:")
print(f"  Best Validation Accuracy: {max(baseline_val_acc):.4f}")
print(f"  Best Validation Loss: {min(baseline_val_loss):.4f}")
print(f"  Final Validation Accuracy: {baseline_val_acc[-1]:.4f}")
print(f"  Final Validation Loss: {baseline_val_loss[-1]:.4f}")
print(f"\nConvergence:")
best_epoch_baseline = np.argmax(baseline_val_acc) + 1
print(f"  Epoch of Best Val Acc: {best_epoch_baseline}")
print(f"\nGeneralization Gap:")
gap_baseline = baseline_train_acc[-1] - baseline_val_acc[-1]
print(f"  Train-Val Accuracy Gap: {gap_baseline:.4f}")
print(f"\nTest Set Performance:")
print(f"  Test Accuracy: {test_accuracy:.4f}")

print("\n" + "-"*70)
print("OPTIMIZED MODEL")
print("-"*70)
print(f"Total Epochs Trained: {len(optimized_train_acc)}")
print(f"\nTraining Performance:")
print(f"  Final Training Accuracy: {optimized_train_acc[-1]:.4f}")
print(f"  Final Training Loss: {optimized_train_loss[-1]:.4f}")
print(f"\nValidation Performance:")
print(f"  Best Validation Accuracy: {max(optimized_val_acc):.4f}")
print(f"  Best Validation Loss: {min(optimized_val_loss):.4f}")
print(f"  Final Validation Accuracy: {optimized_val_acc[-1]:.4f}")
print(f"  Final Validation Loss: {optimized_val_loss[-1]:.4f}")
print(f"\nConvergence:")
best_epoch_optimized = np.argmax(optimized_val_acc) + 1
print(f"  Epoch of Best Val Acc: {best_epoch_optimized}")
print(f"\nGeneralization Gap:")
gap_optimized = optimized_train_acc[-1] - optimized_val_acc[-1]
print(f"  Train-Val Accuracy Gap: {gap_optimized:.4f}")
print(f"\nTest Set Performance:")
print(f"  Test Accuracy: {optimized_acc:.4f}")

print("\n" + "-"*70)
print("IMPROVEMENT ANALYSIS")
print("-"*70)
print(f"\nAccuracy Improvements:")
print(f"  Test Accuracy: {test_accuracy:.4f} → {optimized_acc:.4f} ({(optimized_acc-test_accuracy)*100:+.2f}%)")
print(f"  Best Val Accuracy: {max(baseline_val_acc):.4f} → {max(optimized_val_acc):.4f} ({(max(optimized_val_acc)-max(baseline_val_acc))*100:+.2f}%)")
print(f"\nLoss Improvements:")
print(f"  Best Val Loss: {min(baseline_val_loss):.4f} → {min(optimized_val_loss):.4f} ({(min(optimized_val_loss)-min(baseline_val_loss))*100:+.2f}%)")
print(f"\nGeneralization:")
print(f"  Train-Val Gap: {gap_baseline:.4f} → {gap_optimized:.4f} ({(gap_optimized-gap_baseline)*100:+.2f}%)")
gap_improvement = "Better" if gap_optimized < gap_baseline else "Worse"
print(f"  Generalization: {gap_improvement}")
print(f"\nConvergence Speed:")
print(f"  Epochs to Best: {best_epoch_baseline} → {best_epoch_optimized} ({best_epoch_optimized-best_epoch_baseline:+d})")
print(f"  Total Epochs: {len(baseline_train_acc)} → {len(optimized_train_acc)} ({len(optimized_train_acc)-len(baseline_train_acc):+d})")

print("\n" + "="*70)
print("✓ Training dynamics analysis complete!")
print("="*70)

In [ ]:
# Comprehensive Performance Summary Visualization
fig = plt.figure(figsize=(18, 10))

# Plot 1: Overall Accuracy Comparison
ax1 = plt.subplot(2, 3, 1)
models = ['Baseline', 'Optimized']
test_accs = [test_accuracy, optimized_acc]
val_accs = [max(baseline_val_acc), max(optimized_val_acc)]

x = np.arange(len(models))
width = 0.35

bars1 = ax1.bar(x - width/2, test_accs, width, label='Test Accuracy', 
                alpha=0.8, color='steelblue', edgecolor='black', linewidth=2)
bars2 = ax1.bar(x + width/2, val_accs, width, label='Best Val Accuracy',
                alpha=0.8, color='coral', edgecolor='black', linewidth=2)

ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Overall Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim([0, 1])

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 2: Loss Comparison
ax2 = plt.subplot(2, 3, 2)
losses = [min(baseline_val_loss), min(optimized_val_loss)]

bars = ax2.bar(models, losses, color=['steelblue', 'coral'], 
               alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title('Best Validation Loss Comparison', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 3: Convergence Speed
ax3 = plt.subplot(2, 3, 3)
convergence = [best_epoch_baseline, best_epoch_optimized]

bars = ax3.bar(models, convergence, color=['steelblue', 'coral'],
               alpha=0.8, edgecolor='black', linewidth=2)
ax3.set_ylabel('Epochs', fontsize=12, fontweight='bold')
ax3.set_title('Epochs to Best Validation Accuracy', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 4: Train-Val Gap (Generalization)
ax4 = plt.subplot(2, 3, 4)
gaps = [gap_baseline, gap_optimized]

bars = ax4.bar(models, gaps, color=['steelblue', 'coral'],
               alpha=0.8, edgecolor='black', linewidth=2)
ax4.set_ylabel('Accuracy Gap', fontsize=12, fontweight='bold')
ax4.set_title('Train-Val Generalization Gap\n(Lower is Better)', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')
ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}',
            ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=10, fontweight='bold')

# Plot 5: Improvement Metrics
ax5 = plt.subplot(2, 3, 5)
improvements = {
    'Test Acc': (optimized_acc - test_accuracy) * 100,
    'Val Acc': (max(optimized_val_acc) - max(baseline_val_acc)) * 100,
    'Val Loss': (min(optimized_val_loss) - min(baseline_val_loss)) * 100,
    'Gen Gap': (gap_optimized - gap_baseline) * 100
}

metric_names = list(improvements.keys())
metric_values = list(improvements.values())
colors_imp = ['green' if v > 0 else 'red' for v in metric_values[:2]] + \
             ['green' if v < 0 else 'red' for v in metric_values[2:]]

bars = ax5.barh(metric_names, metric_values, color=colors_imp, 
                alpha=0.7, edgecolor='black', linewidth=2)
ax5.set_xlabel('Improvement (%)', fontsize=12, fontweight='bold')
ax5.set_title('Performance Improvements\n(Optimized - Baseline)', fontsize=14, fontweight='bold')
ax5.grid(True, alpha=0.3, axis='x')
ax5.axvline(x=0, color='black', linestyle='-', linewidth=1)

for i, (bar, val) in enumerate(zip(bars, metric_values)):
    ax5.text(val, bar.get_y() + bar.get_height()/2.,
            f'{val:+.2f}%',
            ha='left' if val > 0 else 'right',
            va='center', fontsize=10, fontweight='bold')

# Plot 6: Summary Text
ax6 = plt.subplot(2, 3, 6)
ax6.axis('off')

summary_text = f"""
OPTIMIZATION IMPACT SUMMARY

✓ Test Accuracy Improvement:
  {test_accuracy:.4f} → {optimized_acc:.4f}
  Δ = {(optimized_acc - test_accuracy)*100:+.2f}%

✓ Validation Accuracy Improvement:
  {max(baseline_val_acc):.4f} → {max(optimized_val_acc):.4f}
  Δ = {(max(optimized_val_acc) - max(baseline_val_acc))*100:+.2f}%

✓ Loss Reduction:
  {min(baseline_val_loss):.4f} → {min(optimized_val_loss):.4f}
  Δ = {(min(optimized_val_loss) - min(baseline_val_loss))*100:+.2f}%

✓ Convergence Speed:
  {best_epoch_baseline} → {best_epoch_optimized} epochs
  Δ = {best_epoch_optimized - best_epoch_baseline:+d} epochs

✓ Generalization Gap:
  {gap_baseline:.4f} → {gap_optimized:.4f}
  Δ = {(gap_optimized - gap_baseline)*100:+.2f}%

{'✓ Better Generalization' if gap_optimized < gap_baseline else '⚠ Worse Generalization'}
{'✓ Faster Convergence' if best_epoch_optimized < best_epoch_baseline else '⚠ Slower Convergence'}
{'✓ Improved Performance' if optimized_acc > test_accuracy else '⚠ No Improvement'}
"""

ax6.text(0.1, 0.9, summary_text, transform=ax6.transAxes,
         fontsize=11, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))

plt.suptitle('Comprehensive Performance Comparison: Baseline vs Optimized', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

print("\n✓ Comprehensive performance summary complete!")

#### **Cross-Validation Experiments**

Comprehensive model evaluation using multiple cross-validation strategies to assess generalization:

---

### **Experiment 1: K-Fold Window-Based Cross-Validation**

**Configuration:**
- K=11 folds
- Split by temporal window segments (not subjects)
- Random shuffle before folding
- Uses cyclic iterator for balanced fold distribution

**Purpose:**
- Assess model generalization to unseen temporal windows
- Tests ability to classify different time segments
- Faster than subject-based CV
- Subject data can appear in both train and test sets

**Characteristics:**
- May overestimate performance (data leakage from same subject)
- Good for assessing temporal pattern learning
- Less representative of real-world deployment

---

### **Experiment 2: Leave-One-Subject-Out Cross-Validation (LOSOCV)**

**Configuration:**
- Folds equal to number of unique subjects
- Each fold: one subject for testing, all others for training
- Windows from same subject grouped together
- No data leakage between train/test

**Purpose:**
- **Gold standard** for subject-independent evaluation
- Tests true generalization to new, unseen individuals
- Most realistic clinical deployment scenario
- Accounts for inter-subject variability

**Clinical Relevance:**
- Simulates real diagnostic workflow
- Model must generalize to patients not in training set
- Identifies subject-specific overfitting
- More conservative performance estimate

---

### **Evaluation Metrics**

Both experiments report:
- **Mean accuracy** across all folds
- **Standard deviation** (indicates model stability)
- **Per-fold accuracy** for detailed analysis

**Interpretation:**
- Lower std dev = more stable, reliable model
- Window-based: baseline generalization capability
- Subject-based: clinically relevant performance

---

### **Trade-offs Summary**

| Aspect | Window-Based K-Fold | Leave-One-Subject-Out |
|--------|---------------------|------------------------|
| **Speed** | Faster | Slower (more folds) |
| **Realism** | Less realistic | Highly realistic |
| **Performance** | Often higher | More conservative |
| **Data Leakage** | Possible | None |
| **Clinical Value** | Limited | High |
| **Use Case** | Quick validation | Final evaluation |

Both approaches validate model robustness from complementary perspectives and help identify different types of overfitting.

In [ ]:
import random

# map params to integers where needed
params = {'batch_size': 64, 'cnn_dense': 256, 'cnn_dropout': np.float64(0.3571401842400106), 'cnn_kernel_size_1': 5, 'cnn_kernel_size_2': 3, 'cnn_kernels_1': 32, 'cnn_kernels_2': 16, 'learning_rate': np.float64(1.8587640600578385e-05), 'lstm_dense': 64, 'lstm_hidden_size': 96, 'lstm_layers': 2, 'optimizer': 'sgd'}

criterion = nn.CrossEntropyLoss()
unique_subjects = list(df['Window'].unique())
accs = []
variances = []
batch_size = params['batch_size']

K_FOLDS = 11
fold_size = len(unique_subjects) // K_FOLDS

random.shuffle(unique_subjects)

cyclic = itertools.cycle(unique_subjects)
batched_cyclic = batched(cyclic, n=fold_size)
folds = itertools.islice(batched_cyclic, K_FOLDS)
    
for i, fold in enumerate(folds):
    print(f"Starting fold {i + 1}/{K_FOLDS}")

    train_df = df[~df['Window'].isin(fold)]
    test_df   = df[df['Window'].isin(fold)]

    train_loader = get_dataset(train_df, batch_size=batch_size)
    test_loader  = get_dataset(test_df, batch_size=batch_size)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=100,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=15
    )

    acc, *_ = get_validation(model, test_loader, device)
    accs.append(acc)

    print(f"Fold {i + 1} Accuracy:", acc)

print(f"k-Fold CV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:
import random

params = readable

unique_samples = list(df['Window'].unique())
subject_count = df['ID'].nunique()
accs = []
variances = []
batch_size = params['batch_size']

fold_size = len(unique_samples) // subject_count
random.shuffle(unique_samples)

cyclic = itertools.cycle(unique_samples)
batched_cyclic = batched(cyclic, n=fold_size)
folds = itertools.islice(batched_cyclic, subject_count)

for i, fold in enumerate(folds):
    print(f"[{get_timestamp()}] Starting fold {i + 1}/{subject_count}")

    train_df = df[~df['Window'].isin(fold)]
    test_df   = df[df['Window'].isin(fold)]

    train_loader = get_dataset(train_df, is_train=True, batch_size=batch_size)
    test_loader  = get_dataset(test_df, is_train=False, batch_size=batch_size)
    model, criterion, optimizer = get_model(params)

    # Train with modest epochs; early stopping inside fit handles rest
    history = model.fit(
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=60,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        patience=15
    )

    acc, *_ = get_validation(model, test_loader, device)
    accs.append(acc)

    print(f"Fold {i + 1} Accuracy:", acc)

print(f"LOSOCV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:

params = {'batch_size': 48, 'cnn_dense': 64, 'cnn_dropout': np.float64(0.5130373328759822), 'cnn_kernel_size_1': 5, 'cnn_kernel_size_2': 5, 'cnn_kernels_1': 16, 'cnn_kernels_2': 16, 'learning_rate': np.float64(0.000462968156587811), 'lstm_dense': 256, 'lstm_hidden_size': 128, 'lstm_layers': 1, 'optimizer': 'adam'}

df = pd.read_csv("./processed_clustered_adhdata.csv")

frequency_count = len(df['Frequency'].unique())
window_count = len(df['Window'].unique())
numeric_df = df.drop(['ID', 'Window'], axis=1)

display(df.head())

# shape: (windows, frequencies, electrodes)
full_ndarray = numeric_df.values.reshape((window_count, frequency_count, numeric_df.shape[1]))

X = full_ndarray[:, :, 2:]     # drop ID/Class columns
y = full_ndarray[:, 0, 0]      # class label is repeated across freq rows

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Add channel dimension (N, 1, freq, electrodes)
X_train = X_train[..., np.newaxis]   # (N, freq, electrodes, 1)
X_test  = X_test[...,  np.newaxis]

print(X_train.shape)

print("Train shape:", X_train.shape)  # (N, freq, electrodes, 1)
train_ds = EEGDataset(X_train, y_train)
test_ds  = EEGDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)

unique_samples = list(df['Window'].unique())
subject_count = df['ID'].nunique()
accs = []
variances = []
batch_size = params['batch_size']

model, criterion, optimizer = get_model(params)

# Train with modest epochs; early stopping inside fit handles rest
history = model.fit(
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=60,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    patience=15
)

acc, *_ = get_validation(model, test_loader, device)
accs.append(acc)

print(f"Fold {i + 1} Accuracy:", acc)

print(f"LOSOCV Mean Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")

In [ ]:
acc, report, matrix = get_validation(model, test_loader, device)

print("\nval Accuracy:", acc)
print(report)
print(matrix)